In [38]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from skg import nsphere_fit
import scipy.io as sio
import json
from tqdm import tqdm
import os
import sys
import shutil
import seaborn as sns

%matplotlib qt

# matplotlib style = BLACK BACKGROUND, WHITE TEXT and AXES

# reset to default
plt.rcdefaults()
plt.style.use('dark_background')
plt.rcParams['axes.facecolor'] = 'black'
plt.rcParams['axes.edgecolor'] = 'white'
plt.rcParams['axes.labelcolor'] = 'white'

DATA_DIR = '../data/data_06-09-2024'
PROCESSED_DATA_DIR = '../processed_data/data_06-09-2024'

DATA_CLASS = 'Band'
ignore_flags = ['.DS_Store']

if not os.path.exists(PROCESSED_DATA_DIR):
    os.makedirs(PROCESSED_DATA_DIR)

In [39]:
PHASE1_DURATION = 3600
PHASE2_DURATION = 1800
PHASE3_DURATION = 3600

# Arena parameters
arena_radius = 75 # mm

# Tracking parameters
MAX_ALLOWED_SPEED = 50 # mm/s
ENCOUNTER_DISTANCE = 2 * 5 # mm (body length x 5)

recalculate = False
test_if_loading_works = False

## SUPPORTING FUNCTIONS

In [40]:
def select_points_on_image(image, n_points, instructions=None):
    # Select n_points on the image using OpenCV
    points = []
    image = np.ascontiguousarray(image)
    
    if instructions is  None:
        instructions = f'Select {n_points} points on the image and press Enter to continue'
    else:
        instructions = f'{instructions} and press Enter to continue'

    def select_point(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            if len(points) < n_points:
                points.append((x, y))
            else:
                return
        elif event == cv2.EVENT_RBUTTONDOWN:
            if len(points) > 0:
                points.pop()
        
        # Draw a circle at the selected points
        # make a copy of the image to draw the points
        point_image = np.copy(image)
        for point in points:
            cv2.circle(point_image, point, 5, (255, 255, 255), -1)
            cv2.putText(point_image, f'({point[0]}, {point[1]})', point, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.imshow(instructions, point_image)
                

    cv2.namedWindow(instructions)
    cv2.setMouseCallback(instructions, select_point)
    cv2.imshow(instructions, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return points

In [41]:
def get_ring(outer_ring, inner_ring):

    # fit a circle to the points
    outer_radius, outer_center = nsphere_fit(outer_ring)
    inner_radius, inner_center = nsphere_fit(inner_ring)

    print(f'Outer radius: {outer_radius}, Outer center: {outer_center}')
    print(f'Inner radius: {inner_radius}, Inner center: {inner_center}')

    # get the average center
    center = (outer_center + inner_center) / 2

    # get the average radius
    radius = (outer_radius + inner_radius) / 2

    # get the average trail width
    trail_width = outer_radius - inner_radius

    # ring properties
    ring_props = {
        'outer_radius': outer_radius,
        'outer_center': outer_center.tolist(),
        'inner_radius': inner_radius,
        'inner_center': inner_center.tolist(),
        'center': center.tolist(),
        'radius': radius,
        'trail_width': trail_width,
    }
    return ring_props

def draw_ring(ring_props, bg=None, axis=None, color='r', linestyle='-', filled=False, alpha=1):
    
    # draw the estimated circles
    if axis is None:
        fig, ax = plt.subplots()
    else:
        ax = axis

    if bg is not None:
        ax.imshow(bg, cmap='gray')

    if filled:
        circ_inner = plt.Circle(ring_props['center'], ring_props['radius']-ring_props['trail_width']/2, color='black', fill=True, linestyle=linestyle)
        circ_outer = plt.Circle(ring_props['center'], ring_props['radius']+ring_props['trail_width']/2, color=color, fill=True, linestyle=linestyle, alpha=alpha)
    else:
        circ_inner = plt.Circle(ring_props['center'], ring_props['radius']-ring_props['trail_width']/2, color=color, fill=False, linestyle=linestyle)
        circ_outer = plt.Circle(ring_props['center'], ring_props['radius']+ring_props['trail_width']/2, color=color, fill=False, linestyle=linestyle)
    ax.add_artist(circ_outer)
    ax.add_artist(circ_inner)

    if axis is None:
        ax.set_aspect('equal')
        plt.title('Estimated ring')
        plt.show()

def ring_coordinates(pos,y_props,body_width_adjustment=0.):
    # get the angle of the point
    position = np.arctan2(pos[1]-y_props['center'][1], pos[0]-y_props['center'][0])
    position = position % (2*np.pi)
    # get the distance from the trail
    dist = np.sqrt((pos[0]-y_props['center'][0])**2 + (pos[1]-y_props['center'][1])**2) - y_props['radius']
    
    ## Note that +ve to -ve means going inside the trail

    # get the distance from the outer ring
    dist_outer = np.sqrt((pos[0]-y_props['outer_center'][0])**2 + (pos[1]-y_props['outer_center'][1])**2) - (y_props['outer_radius']-body_width_adjustment)
    # get the distance from the inner ring
    dist_inner = (y_props['inner_radius']+body_width_adjustment) - np.sqrt((pos[0]-y_props['inner_center'][0])**2 + (pos[1]-y_props['inner_center'][1])**2)
    
    return position, dist, dist_outer, dist_inner

In [42]:
def get_tracklets(files,sf,fps,big_ring,small_ring,expand_ring=1):
    tracklets = []
    for file in tqdm(files, desc='Processing files'):
        df = pd.read_csv(file)
        x = df['pos x'].values
        y = df['pos y'].values
        ori = df['ori'].values
        frame = df.index.values
        id = np.ones_like(frame) * int(file.split('/')[-1].split('.csv')[0].split('fly')[-1])

        # interpolate missing values
        x = np.interp(np.arange(len(x)), np.arange(len(x))[~np.isnan(x)], x[~np.isnan(x)])
        y = np.interp(np.arange(len(y)), np.arange(len(y))[~np.isnan(y)], y[~np.isnan(y)])
        ori = np.interp(np.arange(len(ori)), np.arange(len(ori))[~np.isnan(ori)], ori[~np.isnan(ori)])
        
        # get difference between consecutive frames
        d = np.sqrt(np.diff(x)**2 + np.diff(y)**2)
        # get points where d > max_displacement
        points = np.where(d > MAX_ALLOWED_SPEED*sf/fps)[0]
        start = 0
        for i in points:
            t = pd.DataFrame({'pos x': x[start:i+1], 'pos y': y[start:i+1], 'ori': ori[start:i+1], 'frame': frame[start:i+1], 'id': id[start:i+1]})
            tracklets.append(t)
            start = i+1
        t = pd.DataFrame({'pos x': x[start:], 'pos y': y[start:], 'ori': ori[start:], 'frame': frame[start:], 'id': id[start:]})
        tracklets.append(t)
    
     # add ring and yang coordinates to each tracklet
    i = 0
    for t in tqdm(tracklets, desc='Adding ring coordinates'):
        t['big ring pos'], t['big ring dist'], t['outer big ring dist'], t['inner big ring dist'] = zip(*[ring_coordinates([x, y], big_ring, expand_ring*sf) for x, y in zip(t['pos x'], t['pos y'])])
        t['small ring pos'], t['small ring dist'], t['outer small ring dist'], t['inner small ring dist'] = zip(*[ring_coordinates([x, y], small_ring, expand_ring*sf) for x, y in zip(t['pos x'], t['pos y'])])
        t['track id'] = i
        i += 1

    # combine and save tracklets
    tracklets = pd.concat(tracklets)
    return tracklets

def get_crossings(df, column, threshold=0, entries_only=True, minimum_distance_between_crossings=0):
    data = df[column].values
    if not entries_only:
        crossings = np.where(np.diff((data > threshold).astype(int)) != 0)[0]
    else:
        crossings = np.where(np.diff((data > threshold).astype(int)) < 0)[0] # +ve sign to -ve sign means entering the trail
    # make them integers
    crossings = crossings.astype(int)
    # remove crossings that are too close to each other
    if minimum_distance_between_crossings > 0 and len(crossings) > 1:
        true_crossings = [crossings[0]]
        last_crossing = crossings[0]
        for crossing in crossings[1:]:
            if np.sqrt((df['pos x'].iloc[crossing] - df['pos x'].iloc[last_crossing])**2 + (df['pos y'].iloc[crossing] - df['pos y'].iloc[last_crossing])**2) > minimum_distance_between_crossings:
                true_crossings.append(crossing)
                last_crossing = crossing
        crossings = np.array(true_crossings)
    
    return crossings

def get_encounters(tracklets, ring, eff_distance, fps, sf):

    tracklets = [group for _, group in tracklets.groupby('track id')]

    # find all continuous segments where the fly is within the effective distance
    encounters = []
    for t in tqdm(tracklets, desc='Finding encounters'):
        val = np.abs(np.concatenate([[np.inf], t[f'{ring} ring dist'].values])) < eff_distance
        count = np.cumsum(np.concatenate([np.array([np.nan]),np.diff(val.astype(int))])>0)
        mes = val*count

        # get every continuous segment where the fly is within encounter_distance of the ring circle
        encounters += [group for m, group in t.groupby(mes[1:]) if m != 0]

    # add features to the encounters
    for i, encounter in tqdm(enumerate(encounters), desc='Adding features to encounters'):
        encounter['encounter id'] = i

        # timing features
        encounter['encounter duration'] = (encounter['frame'].max() - encounter['frame'].min()) / fps
        encounter['encounter start'] = encounter['frame'].min()/fps

        # distance features
        encounter['displacement'] = np.sqrt(np.diff(encounter['pos x'])**2 + np.diff(encounter['pos y'])**2).sum()/sf
        encounter['mean speed'] = encounter['displacement'] / encounter['encounter duration']

        # get all center crossings
        center_crossings = get_crossings(encounter, f'{ring} ring dist', entries_only=False, minimum_distance_between_crossings=0.5*sf)
        encounter['num center crossings'] = len(center_crossings)
        encounter['center crossings'] = ",".join(list(center_crossings.astype(int).astype(str)))

        # get all outer ring crossings
        outer_crossings = get_crossings(encounter, f'outer {ring} ring dist', entries_only=True, minimum_distance_between_crossings=0.5*sf)
        encounter['num outer entries'] = len(outer_crossings)
        encounter['outer entries'] = ",".join(list(outer_crossings.astype(int).astype(str)))

        # get all inner ring crossings
        inner_crossings = get_crossings(encounter, f'inner {ring} ring dist', entries_only=True, minimum_distance_between_crossings=0.5*sf)
        encounter['num inner entries'] = len(inner_crossings)
        encounter['inner entries'] = ",".join(list(inner_crossings.astype(int).astype(str)))

        # unwrap the position
        if len(outer_crossings) == 0 and len(inner_crossings) == 0:
            first_crossing = 0
        elif len(outer_crossings) == 0:
            first_crossing = inner_crossings[0]
        elif len(inner_crossings) == 0:
            first_crossing = outer_crossings[0]
        else:
            first_crossing = min(outer_crossings[0], inner_crossings[0])

        encounter['encounter position'] = np.unwrap(encounter[f'{ring} ring pos'])
        encounter['encounter position'] = encounter['encounter position'] - encounter['encounter position'].iloc[first_crossing]

        # if mean distance is negative, flip the position
        if encounter['encounter position'][first_crossing:].mean() < 0:
            encounter['encounter position'] = -encounter['encounter position']

        # scale with radius and scale factor
        encounter['encounter position'] = encounter['encounter position'] * eval(f'{ring}_ring')['radius'] / sf
        
        # get fraction of time spent in the trail
        encounter['fraction in trail'] = (encounter[f'{ring} ring dist'] < eval(f'{ring}_ring')['trail_width']/2).sum() / len(encounter)

        # get max distance along the trail
        encounter['maximum distance'] = encounter['encounter position'].max()

        # get tortuosity
        encounter['inverse tortuosity'] = np.sqrt((encounter['pos x'].iloc[-1] - encounter['pos x'].iloc[0])**2 + (encounter['pos y'].iloc[-1] - encounter['pos y'].iloc[0])**2) / encounter['displacement']

        # band adjusted orientation
        encounter['adjusted orientation'] = (np.unwrap(encounter['ori']) - np.unwrap(encounter[f'{ring} ring pos']-np.pi)) % (2*np.pi)


    # combine all encounters
    encounters = pd.concat(encounters)

    return encounters

# PROCESSING

In [43]:
recalculate = False
test_if_loading_works = False

In [44]:
# find all the files in the data directory (including subdirectories) with Thin-Trails in the path
files = []
for root, _, filenames in os.walk(DATA_DIR):
    full_paths = [os.path.join(root, f) for f in filenames]
    for f in full_paths:
        if DATA_CLASS in f and not any([flag in f for flag in ignore_flags]):
            files.append(f)

print(f'Found {len(files)} files with {DATA_CLASS} in the name')

# get unique experiments
experiments = list(set([f.split('/processed')[0] for f in files]))
print(f'Found {len(experiments)} unique experiments')

for exp in experiments:
    phases = os.listdir(exp+'/processed')
    phases = [p for p in phases if p not in ignore_flags]
    assert len(phases) == 3, f'Expected 3 phases, found {len(phases)}.'
    # create a directory for the experiment in the processed data directory
    os.makedirs(exp.replace(DATA_DIR, PROCESSED_DATA_DIR), exist_ok=True)

Found 114 files with Band in the name
Found 4 unique experiments


In [45]:
# for each experiment, process the data
for exp in experiments:
    print(f'Processing {exp}')
    processed_exp = exp.replace(DATA_DIR, PROCESSED_DATA_DIR)

    phases = os.listdir(exp+'/processed')
    phases = [p for p in phases if p not in ignore_flags]
    phases.sort()

    prefix = phases[0].split('_phase_')[0]

    # define the arena properties 
    if os.path.exists(processed_exp+'/arena.json') and not recalculate:
        if test_if_loading_works:
            with open(processed_exp+'/arena.json', 'r') as f:
                arena = json.load(f)
            print('ARENA WORKING OK |', end=' ')
    else:

        # get phase 2 background image
        p2_bg = sio.loadmat(f'{exp}/processed/{prefix}_phase_2/{prefix}_phase_2-bg.mat')['bg'][0][0].item()[0]

        # get arena margins
        pts = select_points_on_image(p2_bg, 5, instructions='Select 5 points on the margins of the arena')

        radius, center = nsphere_fit(pts)
        print('Center:', center)
        print('Radius:', radius)
        sf = radius / arena_radius

        # save the properties of the arena
        arena = {
            'center': center.tolist(),
            'radius': radius,
            'sf': sf
        }

        with open(processed_exp+'/arena.json', 'w') as f:
            json.dump(arena, f)

    # define the big ring properties
    if os.path.exists(processed_exp+'/big_ring.json') and not recalculate:
        if test_if_loading_works:
            with open(processed_exp+'/big_ring.json', 'r') as f:
                big_ring = json.load(f)
            print('BIG RING WORKING OK', end=' | ')
    else:

        # get the phase 1 background image
        p1_bg = sio.loadmat(f'{exp}/processed/{prefix}_phase_1/{prefix}_phase_1-bg.mat')['bg'][0][0].item()[0]

        # get the big ring
        outer_pts = select_points_on_image(p1_bg, 5, instructions='Select 5 points on the outer ring')
        inner_pts = select_points_on_image(p1_bg, 5, instructions='Select 5 points on the inner ring')

        big_ring = get_ring(outer_pts, inner_pts)
        with open(processed_exp+'/big_ring.json', 'w') as f:
            json.dump(big_ring, f)

    # define the small ring properties
    if os.path.exists(processed_exp+'/small_ring.json') and not recalculate:
        if test_if_loading_works:
            with open(processed_exp+'/small_ring.json', 'r') as f:
                small_ring = json.load(f)
            print('SMALL RING WORKING OK')
    else:
        # get the phase 3 background image
        p3_bg = sio.loadmat(f'{exp}/processed/{prefix}_phase_3/{prefix}_phase_3-bg.mat')['bg'][0][0].item()[0]
        
        # get the small ring
        outer_pts = select_points_on_image(p3_bg, 5, instructions='Select 5 points on the outer ring')
        inner_pts = select_points_on_image(p3_bg, 5, instructions='Select 5 points on the inner ring')

        small_ring = get_ring(outer_pts, inner_pts)
        with open(processed_exp+'/small_ring.json', 'w') as f:
            json.dump(small_ring, f)

Processing ../data/data_06-09-2024/Or42B/Band/20hr-wingless-f-or42b-band/2024-04-18_11-38
Processing ../data/data_06-09-2024/Or42B/Band/23hr-wingless-f-or42b-band/2024-04-18_15-25
Processing ../data/data_06-09-2024/OrCo/Band/19hr-wingless-orco-band/2024-04-17_11-17
Processing ../data/data_06-09-2024/OrCo/Band/20hr-wingless-orcoctrl-band/2024-05-29_11-29


In [46]:
recalculate = False

In [47]:
EXP_DB = pd.DataFrame()

# for each experiment, process the data
for exp in experiments:
    print(f'Processing {exp}')
    print('-'*len(f'Processing {exp}'))

    processed_exp = exp.replace(DATA_DIR, PROCESSED_DATA_DIR)

    phases = os.listdir(exp+'/processed')
    phases = [p for p in phases if p not in ignore_flags]
    phases.sort()

    prefix = phases[0].split('_phase_')[0]

    # get background images
    p1_bg = sio.loadmat(f'{exp}/processed/{prefix}_phase_1/{prefix}_phase_1-bg.mat')['bg'][0][0].item()[0]
    p2_bg = sio.loadmat(f'{exp}/processed/{prefix}_phase_2/{prefix}_phase_2-bg.mat')['bg'][0][0].item()[0]
    p3_bg = sio.loadmat(f'{exp}/processed/{prefix}_phase_3/{prefix}_phase_3-bg.mat')['bg'][0][0].item()[0]

    # define the arena properties 
    with open(processed_exp+'/arena.json', 'r') as f:
        arena = json.load(f)

    center = np.array(arena['center'])
    radius = arena['radius']
    sf = arena['sf']

    with open(processed_exp+'/big_ring.json', 'r') as f:
        big_ring = json.load(f)
    
    with open(processed_exp+'/small_ring.json', 'r') as f:
        small_ring = json.load(f)

    p1_flies =os.listdir(f'{exp}/processed/{prefix}_phase_1/{prefix}_phase_1-trackfeat.csv')
    p2_flies =os.listdir(f'{exp}/processed/{prefix}_phase_2/{prefix}_phase_2-trackfeat.csv')
    p3_flies =os.listdir(f'{exp}/processed/{prefix}_phase_3/{prefix}_phase_3-trackfeat.csv')
    assert len(p1_flies) == len(p2_flies) == len(p3_flies), 'Different number of flies in the phases'

    # get the number of flies
    n_flies = len(p1_flies)
    print(f'Found {n_flies} flies')

    # get the number of frames in each phase
    p1_n_frames = pd.read_csv(f'{exp}/processed/{prefix}_phase_1/{prefix}_phase_1-trackfeat.csv/fly1.csv').shape[0]
    p2_n_frames = pd.read_csv(f'{exp}/processed/{prefix}_phase_2/{prefix}_phase_2-trackfeat.csv/fly1.csv').shape[0]
    p3_n_frames = pd.read_csv(f'{exp}/processed/{prefix}_phase_3/{prefix}_phase_3-trackfeat.csv/fly1.csv').shape[0]
    fps = np.round(np.mean([p1_n_frames / PHASE1_DURATION, p2_n_frames / PHASE2_DURATION, p3_n_frames / PHASE3_DURATION]))
    print(f'FPS: {fps}')

    # create and save metadata
    metadata = {
        'n_flies': n_flies,
        'n_frames1': p1_n_frames,
        'n_frames2': p2_n_frames,
        'n_frames3': p3_n_frames,
        'sf': sf, # scale factor
        'fps': fps,
        'data_directory': exp,
        'processed_data_directory': processed_exp,
        'arena': processed_exp+'/arena.json',
        'big_ring': processed_exp+'/big_ring.json',
        'small_ring': processed_exp+'/small_ring.json',
        'starvation': exp.split('/')[-2].split('-')[0],
        'wings?': exp.split('/')[-2].split('-')[1],
        'genotype': exp.split('/')[-2].split('-')[2],
        'date': exp.split('/')[-1].split('_')[0],
        'time': exp.split('/')[-1].split('_')[1],
    }

    # add metadata to the database
    EXP_DB = pd.concat([EXP_DB, pd.DataFrame(metadata, index=[0])])

    phase1_files = [f'{exp}/processed/{prefix}_phase_1/{prefix}_phase_1-trackfeat.csv/fly{i+1}.csv' for i in range(n_flies)]
    phase2_files = [f'{exp}/processed/{prefix}_phase_2/{prefix}_phase_2-trackfeat.csv/fly{i+1}.csv' for i in range(n_flies)]
    phase3_files = [f'{exp}/processed/{prefix}_phase_3/{prefix}_phase_3-trackfeat.csv/fly{i+1}.csv' for i in range(n_flies)]

    # get tracklets if they exist
    if os.path.exists(processed_exp+'/phase1_tracklets.csv') and os.path.exists(processed_exp+'/phase2_tracklets.csv') and os.path.exists(processed_exp+'/phase3_tracklets.csv') and not recalculate:
        if test_if_loading_works:
            p1_tracklets = pd.read_csv(processed_exp+'/phase1_tracklets.csv')
            p2_tracklets = pd.read_csv(processed_exp+'/phase2_tracklets.csv')
            p3_tracklets = pd.read_csv(processed_exp+'/phase3_tracklets.csv')
            del p1_tracklets, p2_tracklets, p3_tracklets
    else:
        p1_tracklets = get_tracklets(phase1_files,sf,fps,big_ring,small_ring)
        p2_tracklets = get_tracklets(phase2_files,sf,fps,big_ring,small_ring)
        p3_tracklets = get_tracklets(phase3_files,sf,fps,big_ring,small_ring)
        p1_tracklets.to_csv(processed_exp+'/phase1_tracklets.csv', index=False)
        p2_tracklets.to_csv(processed_exp+'/phase2_tracklets.csv', index=False)
        p3_tracklets.to_csv(processed_exp+'/phase3_tracklets.csv', index=False)

    # get encounters if they exist
    if os.path.exists(processed_exp+'/p1_big_ring_encounters.csv') and not recalculate:
        if test_if_loading_works:
            p1_big_ring_encounters = pd.read_csv(processed_exp+'/p1_big_ring_encounters.csv')
            del p1_big_ring_encounters
    else:
        p1_tracklets = pd.read_csv(processed_exp+'/phase1_tracklets.csv')
        p1_big_ring_encounters = get_encounters(p1_tracklets, 'big', ENCOUNTER_DISTANCE*sf + big_ring['trail_width']/2, fps, sf)
        p1_big_ring_encounters.to_csv(processed_exp+'/p1_big_ring_encounters.csv', index=False)

    if os.path.exists(processed_exp+'/p2_big_ring_encounters.csv') and not recalculate:
        if test_if_loading_works:
            p2_big_ring_encounters = pd.read_csv(processed_exp+'/p2_big_ring_encounters.csv')
            del p2_big_ring_encounters
    else:
        p2_tracklets = pd.read_csv(processed_exp+'/phase2_tracklets.csv')
        p2_big_ring_encounters = get_encounters(p2_tracklets, 'big', ENCOUNTER_DISTANCE*sf + big_ring['trail_width']/2, fps, sf)
        p2_big_ring_encounters.to_csv(processed_exp+'/p2_big_ring_encounters.csv', index=False)

    if os.path.exists(processed_exp+'/p2_small_ring_encounters.csv') and not recalculate:
        if test_if_loading_works:
            p2_small_ring_encounters = pd.read_csv(processed_exp+'/p2_small_ring_encounters.csv')
            del p2_small_ring_encounters
    else:
        p2_tracklets = pd.read_csv(processed_exp+'/phase2_tracklets.csv')
        p2_small_ring_encounters = get_encounters(p2_tracklets, 'small', ENCOUNTER_DISTANCE*sf + small_ring['trail_width']/2, fps, sf)
        p2_small_ring_encounters.to_csv(processed_exp+'/p2_small_ring_encounters.csv', index=False)

    if os.path.exists(processed_exp+'/p3_small_ring_encounters.csv') and not recalculate:
        if test_if_loading_works:
            p3_small_ring_encounters = pd.read_csv(processed_exp+'/p3_small_ring_encounters.csv')
            del p3_small_ring_encounters
    else:
        p3_tracklets = pd.read_csv(processed_exp+'/phase3_tracklets.csv')
        p3_small_ring_encounters = get_encounters(p3_tracklets, 'small', ENCOUNTER_DISTANCE*sf + small_ring['trail_width']/2, fps, sf)
        p3_small_ring_encounters.to_csv(processed_exp+'/p3_small_ring_encounters.csv', index=False)

    print('Done\n-------------------\n\n')

EXP_DB.sort_values(by=['genotype', 'starvation', 'date', 'time'], inplace=True)
EXP_DB.reset_index(drop=True, inplace=True)

# save the database
EXP_DB.to_csv(PROCESSED_DATA_DIR+'/band-experiment_database.csv', index=False)

Processing ../data/data_06-09-2024/Or42B/Band/20hr-wingless-f-or42b-band/2024-04-18_11-38
-----------------------------------------------------------------------------------------
Found 5 flies
FPS: 10.0
Done
-------------------


Processing ../data/data_06-09-2024/Or42B/Band/23hr-wingless-f-or42b-band/2024-04-18_15-25
-----------------------------------------------------------------------------------------
Found 5 flies
FPS: 10.0
Done
-------------------


Processing ../data/data_06-09-2024/OrCo/Band/19hr-wingless-orco-band/2024-04-17_11-17
-------------------------------------------------------------------------------------
Found 5 flies
FPS: 10.0
Done
-------------------


Processing ../data/data_06-09-2024/OrCo/Band/20hr-wingless-orcoctrl-band/2024-05-29_11-29
-----------------------------------------------------------------------------------------
Found 4 flies
FPS: 10.0
Done
-------------------




# ANALYSIS

In [48]:
# load experiment database
EXP_DB = pd.read_csv(PROCESSED_DATA_DIR+'/band-experiment_database.csv')

In [49]:
EXP_DB

,n_flies,n_frames1,n_frames2,n_frames3,sf,fps,data_directory,processed_data_directory,arena,big_ring,small_ring,starvation,wings?,genotype,date,time
0,5,36000,18000,35954,5.535856,10.0,../data/data_06-09-2024/Or42B/Band/20hr-wingle...,../processed_data/data_06-09-2024/Or42B/Band/2...,../processed_data/data_06-09-2024/Or42B/Band/2...,../processed_data/data_06-09-2024/Or42B/Band/2...,../processed_data/data_06-09-2024/Or42B/Band/2...,20hr,wingless,f,2024-04-18,11-38
1,5,36000,18000,35924,5.525171,10.0,../data/data_06-09-2024/Or42B/Band/23hr-wingle...,../processed_data/data_06-09-2024/Or42B/Band/2...,../processed_data/data_06-09-2024/Or42B/Band/2...,../processed_data/data_06-09-2024/Or42B/Band/2...,../processed_data/data_06-09-2024/Or42B/Band/2...,23hr,wingless,f,2024-04-18,15-25
2,5,36000,18000,35948,5.518664,10.0,../data/data_06-09-2024/OrCo/Band/19hr-wingles...,../processed_data/data_06-09-2024/OrCo/Band/19...,../processed_data/data_06-09-2024/OrCo/Band/19...,../processed_data/data_06-09-2024/OrCo/Band/19...,../processed_data/data_06-09-2024/OrCo/Band/19...,19hr,wingless,orco,2024-04-17,11-17
3,4,36000,18000,35967,9.287088,10.0,../data/data_06-09-2024/OrCo/Band/20hr-wingles...,../processed_data/data_06-09-2024/OrCo/Band/20...,../processed_data/data_06-09-2024/OrCo/Band/20...,../processed_data/data_06-09-2024/OrCo/Band/20...,../processed_data/data_06-09-2024/OrCo/Band/20...,20hr,wingless,orcoctrl,2024-05-29,11-29


### PLOT DENSITY

In [57]:
def plot_density(index, phase, ring, color='white', linestyle='-',show=False):
    if not os.path.exists(EXP_DB['processed_data_directory'].iloc[index] + '/figures'):
        os.makedirs(EXP_DB['processed_data_directory'].iloc[index] + '/figures')
    # get the density of tracklets
    tracklets = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/phase{phase}_tracklets.csv')
    # create a logscale hexbin plot of the tracklets pos x and pos y
    plt.figure()
    plt.hexbin(tracklets['pos x'], tracklets['pos y'], gridsize=50, cmap='inferno', bins='log')
    ring_props = json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r'))
    draw_ring(ring_props, bg=None, axis=plt.gca(), color=color, linestyle=linestyle)
    plt.gca().set_aspect('equal')
    plt.title(f'Phase {phase}')
    plt.axis('off')
    plt.savefig(f'{EXP_DB["processed_data_directory"].iloc[index]}/figures/phase{phase}_density.png', dpi=300)
    if show:
        plt.show()
    plt.close()

for index in range(EXP_DB.shape[0]):
    plot_density(index, 1, 'big')
    plot_density(index, 2, 'big', color='gray', linestyle='--')
    plot_density(index, 3, 'small')

In [58]:
index = 2
phase = 1
ring = 'big'

tracklets = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/phase{phase}_tracklets.csv')
tracklets = [group for _, group in tracklets.groupby('track id')]
encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
encounters = [group for _, group in encounters.groupby('encounter id')]
# remove encounters with no entries
encounters = [e for e in encounters if e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] > 0]

# plot all the tracklets
plt.figure()
for t in tracklets:
    plt.plot(t['pos x'], t['pos y'], color='white', alpha=1, linewidth=0.1)
plt.gca().set_aspect('equal')
plt.axis('off')
plt.show()

# plot all the encounters
plt.figure()
for e in encounters:
    plt.plot(e['pos x'], e['pos y'], color='white', alpha=1, linewidth=0.1)
draw_ring(json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r')), bg=None, axis=plt.gca(), color='red')
plt.gca().set_aspect('equal')
plt.axis('off')
plt.show()


In [59]:
encounters = [e for e in encounters if e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] > 5]

### PLOT EXAMPLE ENCOUNTER

In [61]:
# plot a random encounter
plt.figure()
# e = [e for e in encounters if e['encounter id'].iloc[0] == 10][0]
e = encounters[np.random.randint(len(encounters))]
plt.plot(e['pos x'], e['pos y'], color='white', alpha=1, linewidth=1)
# plot the center crossings
center_crossings = e['center crossings'].iloc[0].split(',')
center_crossings = np.array([int(c) for c in center_crossings])
plt.scatter(e['pos x'].iloc[center_crossings], e['pos y'].iloc[center_crossings], color='gray', s=20, zorder=10, label='Center crossings')
plt.plot([e['pos x'].iloc[center_crossings-1], e['pos x'].iloc[center_crossings]], [e['pos y'].iloc[center_crossings-1], e['pos y'].iloc[center_crossings]], color='gray', linewidth=3)
plt.plot([e['pos x'].iloc[center_crossings], e['pos x'].iloc[center_crossings+1]], [e['pos y'].iloc[center_crossings], e['pos y'].iloc[center_crossings+1]], color='gray', linewidth=3)
# plot the outer ring entries
outer_entries = e['outer entries'].iloc[0].split(',')
outer_entries = np.array([int(c) for c in outer_entries])
plt.scatter(e['pos x'].iloc[outer_entries], e['pos y'].iloc[outer_entries], color='blue', s=20, zorder=10, label='Outer entries')
plt.plot([e['pos x'].iloc[outer_entries-1], e['pos x'].iloc[outer_entries]], [e['pos y'].iloc[outer_entries-1], e['pos y'].iloc[outer_entries]], color='blue', linewidth=3)
plt.plot([e['pos x'].iloc[outer_entries], e['pos x'].iloc[outer_entries+1]], [e['pos y'].iloc[outer_entries], e['pos y'].iloc[outer_entries+1]], color='blue', linewidth=3)
# plot the inner ring entries
inner_entries = e['inner entries'].iloc[0].split(',')
inner_entries = np.array([int(c) for c in inner_entries])
plt.scatter(e['pos x'].iloc[inner_entries], e['pos y'].iloc[inner_entries], color='green', s=20, zorder=10, label='Inner entries')
plt.plot([e['pos x'].iloc[inner_entries-1], e['pos x'].iloc[inner_entries]], [e['pos y'].iloc[inner_entries-1], e['pos y'].iloc[inner_entries]], color='green', linewidth=3)
plt.plot([e['pos x'].iloc[inner_entries], e['pos x'].iloc[inner_entries+1]], [e['pos y'].iloc[inner_entries], e['pos y'].iloc[inner_entries+1]], color='green', linewidth=3)
# mark the start and end
plt.scatter(e['pos x'].iloc[0], e['pos y'].iloc[0], color='white', s=50, zorder=10, marker='x')
plt.text(e['pos x'].iloc[0], e['pos y'].iloc[0], 'Start', color='white', fontsize=12, verticalalignment='bottom', horizontalalignment='right')
plt.scatter(e['pos x'].iloc[-1], e['pos y'].iloc[-1], color='white', s=50, zorder=10, marker='x')
plt.text(e['pos x'].iloc[-1], e['pos y'].iloc[-1], 'End', color='white', fontsize=12, verticalalignment='bottom', horizontalalignment='right')

draw_ring(json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r')), bg=None, axis=plt.gca(), color='red', filled=True, alpha=0.3)

plt.gca().set_aspect('equal')
plt.axis('off')
plt.legend(frameon=False)
plt.show()

# plot the encounter unwrapped
plt.figure()
plt.plot(e['encounter position'], e[f'{ring} ring dist']/sf, color='white', alpha=1, linewidth=1)

# fill between the trail margins
x_min = e['encounter position'].min()
x_max = e['encounter position'].max()
plt.fill_between([x_min, x_max], -big_ring['trail_width']/2/sf, big_ring['trail_width']/2/sf, color='red', alpha=0.3)

# plot the center crossings
center_crossings = e['center crossings'].iloc[0].split(',')
center_crossings = np.array([int(c) for c in center_crossings])
plt.scatter(e['encounter position'].iloc[center_crossings], e[f'{ring} ring dist'].iloc[center_crossings]/sf, color='gray', s=20, zorder=10, label='Center crossings')
plt.plot([e['encounter position'].iloc[center_crossings-1], e['encounter position'].iloc[center_crossings]], [e[f'{ring} ring dist'].iloc[center_crossings-1]/sf, e[f'{ring} ring dist'].iloc[center_crossings]/sf], color='gray', linewidth=3)
plt.plot([e['encounter position'].iloc[center_crossings], e['encounter position'].iloc[center_crossings+1]], [e[f'{ring} ring dist'].iloc[center_crossings]/sf, e[f'{ring} ring dist'].iloc[center_crossings+1]/sf], color='gray', linewidth=3)

# plot the outer ring entries
outer_entries = e['outer entries'].iloc[0].split(',')
outer_entries = np.array([int(c) for c in outer_entries])
plt.scatter(e['encounter position'].iloc[outer_entries], e[f'{ring} ring dist'].iloc[outer_entries]/sf, color='blue', s=20, zorder=10, label='Outer entries')
plt.plot([e['encounter position'].iloc[outer_entries-1], e['encounter position'].iloc[outer_entries]], [e[f'{ring} ring dist'].iloc[outer_entries-1]/sf, e[f'{ring} ring dist'].iloc[outer_entries]/sf], color='blue', linewidth=3)
plt.plot([e['encounter position'].iloc[outer_entries], e['encounter position'].iloc[outer_entries+1]], [e[f'{ring} ring dist'].iloc[outer_entries]/sf, e[f'{ring} ring dist'].iloc[outer_entries+1]/sf], color='blue', linewidth=3)

# plot the inner ring entries
inner_entries = e['inner entries'].iloc[0].split(',')
inner_entries = np.array([int(c) for c in inner_entries])
plt.scatter(e['encounter position'].iloc[inner_entries], e[f'{ring} ring dist'].iloc[inner_entries]/sf, color='green', s=20, zorder=10, label='Inner entries')
plt.plot([e['encounter position'].iloc[inner_entries-1], e['encounter position'].iloc[inner_entries]], [e[f'{ring} ring dist'].iloc[inner_entries-1]/sf, e[f'{ring} ring dist'].iloc[inner_entries]/sf], color='green', linewidth=3)
plt.plot([e['encounter position'].iloc[inner_entries], e['encounter position'].iloc[inner_entries+1]], [e[f'{ring} ring dist'].iloc[inner_entries]/sf, e[f'{ring} ring dist'].iloc[inner_entries+1]/sf], color='green', linewidth=3)

plt.gca().set_aspect('equal')
plt.gca().invert_yaxis()
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_color('black')

plt.xlabel('Position along the band (mm)')
plt.ylabel('Distance from centerline (mm)')
plt.show()

### PLOT ALL ENCOUNTERS

In [62]:
# plot all encounters

def plot_encounters(index, phase, ring, show=False, color='red'):
    encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
    encounters = [group for _, group in encounters.groupby('encounter id')]
    # remove encounters with no entries
    encounters = [e for e in encounters if e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] > 0]

    ring_props = json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r'))

    sf = EXP_DB['sf'].iloc[index].astype(float)

    plt.figure(figsize=(20, 3))
    for e in encounters:
        plt.plot(e['encounter position'], e[f'{ring} ring dist']/sf, color='white', alpha=0.5, linewidth=0.5)

    # get the x-axis limits
    x_min = min([e['encounter position'].min() for e in encounters])
    x_max = max([e['encounter position'].max() for e in encounters])

    # fill the trail
    plt.fill_between([x_min, x_max], -ring_props['trail_width']/2/sf, ring_props['trail_width']/2/sf, color=color, alpha=0.3)
    plt.xlim(x_min, x_max)
    plt.gca().set_aspect('equal')
    plt.gca().invert_yaxis()
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_color('black')
    plt.xlabel('Position along the band (mm)')
    plt.ylabel('Distance from centerline (mm)')
    plt.title(f'Phase {phase} encounters')
    plt.savefig(f'{EXP_DB["processed_data_directory"].iloc[index]}/figures/phase{phase}_{ring}_encounters.png', dpi=300)
    if show:
        plt.show()
    plt.close()

for index in range(EXP_DB.shape[0]):
    plot_encounters(index, 1, 'big')
    plot_encounters(index, 2, 'big', color='gray')
    plot_encounters(index, 3, 'small')


/var/folders/yj/5_b3w2q176x67w94977kdrx80000gn/T/ipykernel_66784/3481961546.py:4: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
/var/folders/yj/5_b3w2q176x67w94977kdrx80000gn/T/ipykernel_66784/3481961546.py:4: DtypeWarning: Columns (22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
/var/folders/yj/5_b3w2q176x67w94977kdrx80000gn/T/ipykernel_66784/3481961546.py:4: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')


### ORCO VS. ORCO CTRL COMPARISONS

#### ENCOUNTERS

In [63]:
index, phase, ring = 2, 1, 'big'

encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
encounters = [group for _, group in encounters.groupby('encounter id')]
# remove encounters with no entries
encounters = [e for e in encounters if e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] > 0]
ring_props = json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r'))

sf = EXP_DB['sf'].iloc[index].astype(float)

plt.figure(figsize=(20, 3))
for e in encounters:
    plt.plot(e['encounter position'], e[f'{ring} ring dist']/sf, color='white', alpha=0.5, linewidth=0.5)

# get the x-axis limits
x_min = min([e['encounter position'].min() for e in encounters])
x_max = max([e['encounter position'].max() for e in encounters])

print('Number of encounters:', len(encounters), 'Number of flies:', EXP_DB['n_flies'].iloc[index])
# fill the trail
plt.fill_between([x_min, x_max], -ring_props['trail_width']/2/sf, ring_props['trail_width']/2/sf, color='red', alpha=0.3)
plt.xlim(x_min, x_max)
plt.gca().set_aspect('equal')
plt.gca().invert_yaxis()
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_color('black')
plt.xlabel('Position along the band (mm)')
plt.ylabel('Distance from centerline (mm)')
plt.title(f'Phase {phase} encounters')
plt.show()

index, phase, ring = 3, 1, 'big'

encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
encounters = [group for _, group in encounters.groupby('encounter id')]
# remove encounters with no entries
encounters = [e for e in encounters if e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] > 0]
ring_props = json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r'))

sf = EXP_DB['sf'].iloc[index].astype(float)

plt.figure(figsize=(20, 3))
for e in encounters:
    plt.plot(e['encounter position'], e[f'{ring} ring dist']/sf, color='white', alpha=0.5, linewidth=0.5)

print('Number of encounters:', len(encounters), 'Number of flies:', EXP_DB['n_flies'].iloc[index])
# fill the trail
plt.fill_between([x_min, x_max], -ring_props['trail_width']/2/sf, ring_props['trail_width']/2/sf, color='red', alpha=0.3)
plt.xlim(x_min, x_max)
plt.gca().set_aspect('equal')
plt.gca().invert_yaxis()
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_color('black')
plt.xlabel('Position along the band (mm)')
plt.ylabel('Distance from centerline (mm)')
plt.title(f'Phase {phase} encounters')
plt.show()

Number of encounters: 311 Number of flies: 5
Number of encounters: 514 Number of flies: 4


/var/folders/yj/5_b3w2q176x67w94977kdrx80000gn/T/ipykernel_66784/92994233.py:35: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')


#### ENCOUNTER HISTOGRAMS

In [64]:
max_value = 10
# plot a histogram of the number of center crossings
plt.figure(figsize=(5, 4))

# orco
index, phase, ring = 2, 1, 'big'
encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
encounters = [group for _, group in encounters.groupby('encounter id')]
# remove encounters with no entries
encounters = [e for e in encounters if e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] > 0]
bins, edges = np.histogram([e['num center crossings'].iloc[0] for e in encounters], bins=np.arange(-0.5, max_value+0.5, 1))
bins = bins / bins.sum()

plt.bar(np.arange(max_value)+0.125, bins, color='white', edgecolor='black', width=0.25, label='Orco-Gal4/UAS-CsChrimson')

# orcoctrl
index, phase, ring = 3, 1, 'big'
encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
encounters = [group for _, group in encounters.groupby('encounter id')]
# remove encounters with no entries
encounters = [e for e in encounters if e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] > 0]
bins, edges = np.histogram([e['num center crossings'].iloc[0] for e in encounters], bins=np.arange(-0.5, max_value+0.5, 1))
bins = bins / bins.sum()

plt.bar(np.arange(max_value)-0.125, bins, color='gray', edgecolor='black', width=0.25, label='Orco-Gal4/+')

plt.xticks(np.arange(0, max_value, 1))
plt.xlabel('Number of center crossings')
plt.ylabel('Fraction of encounters')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.legend(frameon=False)
plt.show()

# plot a histogram of the number of entries
plt.figure(figsize=(5, 4))

# orco
index, phase, ring = 2, 1, 'big'
encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
encounters = [group for _, group in encounters.groupby('encounter id')]
# remove encounters with no entries
encounters = [e for e in encounters if e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] > 0]
bins, edges = np.histogram([e['num outer entries'].iloc[0]+e['num inner entries'].iloc[0] for e in encounters], bins=np.arange(0.5, max_value+0.5, 1))
bins = bins / bins.sum()

plt.bar(np.arange(1,max_value)+0.125, bins, color='white', edgecolor='black', width=0.25, label='Orco-Gal4/UAS-CsChrimson')

# orcoctrl
index, phase, ring = 3, 1, 'big'
encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
encounters = [group for _, group in encounters.groupby('encounter id')]
# remove encounters with no entries
encounters = [e for e in encounters if e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] > 0]
bins, edges = np.histogram([e['num outer entries'].iloc[0]+e['num inner entries'].iloc[0] for e in encounters], bins=np.arange(0.5, max_value+0.5, 1))
bins = bins / bins.sum()

plt.bar(np.arange(1,max_value)-0.125, bins, color='gray', edgecolor='black', width=0.25, label='Orco-Gal4/+')

plt.xticks(np.arange(1, max_value, 1))
plt.xlabel('Number of entries')
plt.ylabel('Fraction of encounters')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.legend(frameon=False)
plt.show()

/var/folders/yj/5_b3w2q176x67w94977kdrx80000gn/T/ipykernel_66784/2712965783.py:18: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
/var/folders/yj/5_b3w2q176x67w94977kdrx80000gn/T/ipykernel_66784/2712965783.py:52: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')


#### ONLY BEST ENCOUNTERS

In [65]:
# show only the encounters with 1 entries
index, phase, ring = 2, 1, 'big'

encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
encounters = [group for _, group in encounters.groupby('encounter id')]
# remove encounters with no entries
encounters = [e for e in encounters if e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] == 1 or e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] == 2]
ring_props = json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r'))

sf = EXP_DB['sf'].iloc[index].astype(float)

plt.figure(figsize=(4, 3))
for e in encounters:
    plt.plot(e['encounter position'], e[f'{ring} ring dist']/sf, color='white', alpha=0.5, linewidth=0.5)

# get the x-axis limits
x_min = min([e['encounter position'].min() for e in encounters])
x_max = max([e['encounter position'].max() for e in encounters])

print('Number of encounters:', len(encounters), 'Number of flies:', EXP_DB['n_flies'].iloc[index])
# fill the trail
plt.fill_between([x_min, x_max], -ring_props['trail_width']/2/sf, ring_props['trail_width']/2/sf, color='red', alpha=0.3)
plt.xlim(x_min, x_max)
plt.gca().set_aspect('equal')
plt.gca().invert_yaxis()
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_color('black')
plt.show()

index, phase, ring = 3, 1, 'big'

encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
encounters = [group for _, group in encounters.groupby('encounter id')]
# remove encounters with no entries
encounters = [e for e in encounters if e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] == 1 or e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] == 2]
ring_props = json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r'))

sf = EXP_DB['sf'].iloc[index].astype(float)

plt.figure(figsize=(4, 3))
for e in encounters:
    plt.plot(e['encounter position'], e[f'{ring} ring dist']/sf, color='gray', alpha=0.5, linewidth=0.5)

print('Number of encounters:', len(encounters), 'Number of flies:', EXP_DB['n_flies'].iloc[index])
# fill the trail
plt.fill_between([x_min, x_max], -ring_props['trail_width']/2/sf, ring_props['trail_width']/2/sf, color='red', alpha=0.3)
plt.xlim(x_min, x_max)
plt.gca().set_aspect('equal')
plt.gca().invert_yaxis()
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_color('black')
plt.show()

Number of encounters: 260 Number of flies: 5
Number of encounters: 501 Number of flies: 4


/var/folders/yj/5_b3w2q176x67w94977kdrx80000gn/T/ipykernel_66784/1079751778.py:33: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')


In [66]:
# show only the encounters with 1 entries
index, phase, ring = 2, 1, 'big'

encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
encounters = [group for _, group in encounters.groupby('encounter id')]
# remove encounters with no entries
encounters = [e for e in encounters if e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] > 2]
ring_props = json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r'))

sf = EXP_DB['sf'].iloc[index].astype(float)

plt.figure(figsize=(3, 10))
for e in encounters:
    plt.plot(e[f'{ring} ring dist']/sf, e['encounter position'], color='white', alpha=0.5, linewidth=0.5)

# get the x-axis limits
y_min = min([e['encounter position'].min() for e in encounters])
y_max = max([e['encounter position'].max() for e in encounters])

print('Number of encounters:', len(encounters), 'Number of flies:', EXP_DB['n_flies'].iloc[index])
# fill the trail
plt.fill_betweenx([y_min, y_max], -ring_props['trail_width']/2/sf, ring_props['trail_width']/2/sf, color='red', alpha=0.3)
plt.ylim(y_min, y_max)
plt.xticks([-15, 0, 15])
plt.gca().set_aspect('equal')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_color('black')
plt.show()

index, phase, ring = 3, 1, 'big'

encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
encounters = [group for _, group in encounters.groupby('encounter id')]
# remove encounters with no entries
encounters = [e for e in encounters if e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] > 2]
ring_props = json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r'))

sf = EXP_DB['sf'].iloc[index].astype(float)

plt.figure(figsize=(3, 10))
for e in encounters:
    plt.plot(e[f'{ring} ring dist']/sf, e['encounter position'], color='white', alpha=0.5, linewidth=0.5)

print('Number of encounters:', len(encounters), 'Number of flies:', EXP_DB['n_flies'].iloc[index])
# fill the trail
plt.fill_betweenx([y_min, y_max], -ring_props['trail_width']/2/sf, ring_props['trail_width']/2/sf, color='red', alpha=0.3)
plt.ylim(y_min, y_max)
plt.xticks([-15, 0, 15])
plt.gca().set_aspect('equal')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_color('black')
plt.show()

Number of encounters: 51 Number of flies: 5
Number of encounters: 13 Number of flies: 4


/var/folders/yj/5_b3w2q176x67w94977kdrx80000gn/T/ipykernel_66784/3396551013.py:33: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')


In [67]:
# show only the encounters with 1 entries
index, phase, ring = 2, 1, 'big'

encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
encounters = [group for _, group in encounters.groupby('encounter id')]
ring_props = json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r'))

sf = EXP_DB['sf'].iloc[index].astype(float)

plt.figure(figsize=(3, 10))
for e in encounters:
    plt.plot(e[f'{ring} ring dist']/sf, e['encounter position'], color='white', alpha=0.5, linewidth=0.5)

# get the x-axis limits
y_min = max([min([e['encounter position'].min() for e in encounters]), -70])
y_max = max([400]+[e['encounter position'].max() for e in encounters])

print('Number of encounters:', len(encounters), 'Number of flies:', EXP_DB['n_flies'].iloc[index])
# fill the trail
plt.fill_betweenx([y_min, y_max], -ring_props['trail_width']/2/sf, ring_props['trail_width']/2/sf, color='red', alpha=0.3)
plt.ylim(y_min, y_max)
plt.xticks([-15, 0, 15])
plt.gca().set_aspect('equal')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_color('black')
plt.show()

# plot a distribution of ring distances
# plot a distance density plot
fig,ax=plt.subplots(1,1,figsize=(5,3))
distances = np.concatenate([e[f'{ring} ring dist'].values/sf for e in encounters])
# plot the density
plt.hist(distances,bins=100,density=True,color='white',alpha=0.5)
# draw the trail
plt.fill_between([-ring_props['trail_width']/2/sf, ring_props['trail_width']/2/sf],0,0.075,color='red',alpha=0.3)
ax.set_xlabel('Distance from the trail (mm)')
ax.set_yticks([])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
plt.tight_layout()
plt.show()

Number of encounters: 583 Number of flies: 5


#### ENCOUNTER PROPERRTIES

In [68]:
# get properties
def get_props(encounters):
    props = {
        'duration (s)': [e['encounter duration'].iloc[0] for e in encounters],
        'inner entries': [e['num inner entries'].iloc[0] for e in encounters],
        'outer entries': [e['num outer entries'].iloc[0] for e in encounters],
        'total entries': [e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] for e in encounters], 
        'center crossings': [e['num center crossings'].iloc[0] for e in encounters],
        'mean speed (mm/s)': [e['mean speed'].iloc[0] for e in encounters],
        'fraction in trail': [e['fraction in trail'].iloc[0] for e in encounters],
        'max distance (mm)': [e['maximum distance'].iloc[0] for e in encounters],
        'inverse tortuosity': [e['inverse tortuosity'].iloc[0] for e in encounters],
        'displacement (mm)': [e['displacement'].iloc[0] for e in encounters],
    }
    return props

def p_value_to_stars(p_value):
    if p_value < 0.0001:
        return '****'
    elif p_value < 0.001:
        return '***'
    elif p_value < 0.01:
        return '**'
    elif p_value < 0.05:
        return '*'
    else:
        return ''

In [72]:


# orco 
# get the interesting encounters
index, phase, ring = 2, 1, 'big'
encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
encounters = [group for _, group in encounters.groupby('encounter id')]
# keep only encounters with more than 2 entries
encounters = [e for e in encounters if e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] > 2]
# get the ring properties
ring_props = json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r'))
orco_props = get_props(encounters)

# orcoctrl
# get the interesting encounters
index, phase, ring = 3, 1, 'big'
encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
encounters = [group for _, group in encounters.groupby('encounter id')]
# keep only encounters with more than 2 entries
encounters = [e for e in encounters if e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] > 2]
# get the ring properties
ring_props = json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r'))
orcoctrl_props = get_props(encounters)

# make a dataframe
orco_df = pd.DataFrame(orco_props)
orco_df['genotype'] = 'Orco-Gal4/\nUAS-CsChrimson'
orcoctrl_df = pd.DataFrame(orcoctrl_props)
orcoctrl_df['genotype'] = 'Orco-Gal4/+'
df = pd.concat([orco_df, orcoctrl_df], axis=0)
# reset the index
df.reset_index(drop=True, inplace=True)


/var/folders/yj/5_b3w2q176x67w94977kdrx80000gn/T/ipykernel_66784/1190072463.py:15: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')


In [78]:
for props in ['duration (s)', 'mean speed (mm/s)', 'fraction in trail', 'max distance (mm)', 'inverse tortuosity', 'displacement (mm)']:
    plt.figure(figsize=(3, 4))
    
    sns.stripplot(x='genotype', y=props, data=df, palette=['white', 'gray'], alpha=1, size=5, hue='genotype')
    # show the mean line
    sns.pointplot(x='genotype', y=props, data=df, color='darkgray', markers='_', errorbar=('ci', 95), capsize=0.1, zorder=10, linewidth=1)

    # add significance and effect size
    from scipy.stats import mannwhitneyu
    p = mannwhitneyu(orco_df[props], orcoctrl_df[props])[1]
    plt.text(0.5, df[props].max()*0.95, p_value_to_stars(p), fontsize=12, horizontalalignment='center')
    plt.text(0.5, df[props].max()*0.9, f'p={p:.3f}', fontsize=8, horizontalalignment='center')


    plt.xticks([0, 1], ['Orco-Gal4/\nUAS-CsChrimson', 'Orco-Gal4/+'])
    plt.ylabel(props)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_color('black')
    plt.tight_layout()
    # turn off the legend
    plt.gca().get_legend().remove()
    plt.show()

for props in ['center crossings', 'total entries']:
    plt.figure(figsize=(3, 4))
    
    sns.histplot(x='genotype', y=props, data=df, palette=['white', 'gray'], alpha=1, hue='genotype', legend=False)

    # add significance and effect size
    from scipy.stats import mannwhitneyu
    p = mannwhitneyu(orco_df[props], orcoctrl_df[props])[1]
    plt.text(0.5, df[props].max()*1.10, p_value_to_stars(p), fontsize=12, horizontalalignment='center')
    plt.text(0.5, df[props].max()*1.05, f'p={p:.3f}', fontsize=8, horizontalalignment='center')


    plt.xticks([0, 1], ['Orco-Gal4/\nUAS-CsChrimson', 'Orco-Gal4/+'])
    plt.ylabel(props)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_color('black')
    plt.tight_layout()
    plt.show()

/Users/neurorishika/mambaforge3/lib/python3.10/site-packages/seaborn/categorical.py:1733: UserWarning: You passed a edgecolor/edgecolors ([(0.6627450980392157, 0.6627450980392157, 0.6627450980392157), (0.6627450980392157, 0.6627450980392157, 0.6627450980392157)]) for an unfilled marker ('_').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  ax.scatter(x, y,
/Users/neurorishika/mambaforge3/lib/python3.10/site-packages/seaborn/categorical.py:1733: UserWarning: You passed a edgecolor/edgecolors ([(0.6627450980392157, 0.6627450980392157, 0.6627450980392157), (0.6627450980392157, 0.6627450980392157, 0.6627450980392157)]) for an unfilled marker ('_').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  ax.scatter(x, y,
/Users/neurorishika/mambaforge3/lib/python3.10/site-packages/seaborn/categorical.py:1733: UserWarning: You passed a edgecolor/edgecolors ([(0.6627450980392157, 0.

#### SPEED AND ANGULAR VELOCITY ANALYSIS

In [596]:
index, phase, ring = 2, 1, 'big'
tracklets = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/phase{phase}_tracklets.csv')
tracklets = [group for _, group in tracklets.groupby('track id')]
fps = EXP_DB['fps'].iloc[index].astype(float)
sf = EXP_DB['sf'].iloc[index].astype(float)
# calculate the speed
for t in tracklets:
    t['speed'] = np.r_[0, np.sqrt(np.diff(t['pos x'])**2 + np.diff(t['pos y'])**2)/sf * fps]
# calculate the angle of movement
for t in tracklets:
    t['angle'] = np.r_[0, np.arctan2(np.diff(t['pos y']), np.diff(t['pos x']))]
    t['angle'] = np.unwrap(t['angle'])
    t['angular speed'] = np.r_[0, np.diff(t['angle']) * fps]
# combine all tracklets
tracklets = pd.concat(tracklets, axis=0)
# make a hexbin plot of the speed as a function of the position
plt.figure()
plt.hexbin(tracklets['pos x'], tracklets['pos y'], C=tracklets['speed'], gridsize=50, cmap='inferno',reduce_C_function=np.mean)
draw_ring(json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r')), bg=None, axis=plt.gca(), color='white')
plt.colorbar(label='Speed (mm/s)')
plt.gca().set_aspect('equal')
plt.axis('off')
plt.show()

index, phase, ring = 3, 1, 'big'
tracklets = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/phase{phase}_tracklets.csv')
tracklets = [group for _, group in tracklets.groupby('track id')]
fps = EXP_DB['fps'].iloc[index].astype(float)
sf = EXP_DB['sf'].iloc[index].astype(float)
# calculate the speed
for t in tracklets:
    t['speed'] = np.r_[0, np.sqrt(np.diff(t['pos x'])**2 + np.diff(t['pos y'])**2)/sf * fps]
# calculate the angle of movement
for t in tracklets:
    t['angle'] = np.r_[0, np.arctan2(np.diff(t['pos y']), np.diff(t['pos x']))]
    t['angle'] = np.unwrap(t['angle'])
    t['angular speed'] = np.r_[0, np.diff(t['angle']) * fps]
# combine all tracklets
tracklets = pd.concat(tracklets, axis=0)
# make a hexbin plot of the speed as a function of the position
plt.figure()
plt.hexbin(tracklets['pos x'], tracklets['pos y'], C=tracklets['speed'], gridsize=50, cmap='inferno',reduce_C_function=np.mean)
draw_ring(json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r')), bg=None, axis=plt.gca(), color='white')
plt.colorbar(label='Speed (mm/s)')
plt.gca().set_aspect('equal')
plt.axis('off')
plt.show()



In [622]:
# encounter density
index, phase, ring = 2, 1, 'big'
encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
encounters = [group for _, group in encounters.groupby('encounter id')]
# remove uninteresting encounters
encounters = [e for e in encounters if e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] > 2]
# join all encounters
encounters = pd.concat(encounters, axis=0)
# get the ring properties
ring_props = json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r'))
# make a hexbin plot of the position of the encounters
plt.figure()
plt.hexbin(encounters['pos x'], encounters['pos y'], gridsize=50, cmap='inferno', bins='log')
draw_ring(json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r')), bg=None, axis=plt.gca(), color='white')
plt.gca().set_aspect('equal')
plt.axis('off')
plt.show()

index, phase, ring = 3, 1, 'big'
encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
encounters = [group for _, group in encounters.groupby('encounter id')]
# remove uninteresting encounters
encounters = [e for e in encounters if e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] > 2]
# join all encounters
encounters = pd.concat(encounters, axis=0)
# get the ring properties
ring_props = json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r'))
# make a hexbin plot of the position of the encounters
plt.figure()
plt.hexbin(encounters['pos x'], encounters['pos y'], gridsize=50, cmap='inferno', bins='log')
draw_ring(json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r')), bg=None, axis=plt.gca(), color='white')
plt.gca().set_aspect('equal')
plt.axis('off')
plt.show()


/var/folders/yj/5_b3w2q176x67w94977kdrx80000gn/T/ipykernel_79510/982533252.py:20: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')


In [956]:
index, phase, ring = 2, 1, 'big'
encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
encounters = [group for _, group in encounters.groupby('encounter id')]
fps = int(EXP_DB['fps'].iloc[index].astype(float))
sf = EXP_DB['sf'].iloc[index].astype(float)
ring_props = json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r'))

# calculate the speed
for e in encounters:
    start_speed = np.sqrt(np.diff(e['pos x'].iloc[:int(fps)])**2 + np.diff(e['pos y'].iloc[:int(fps)])**2)/sf * fps
    start_speed = np.mean(start_speed) if len(start_speed) > 0 else 0
    e['speed'] = np.r_[start_speed, np.sqrt(np.diff(e['pos x'])**2 + np.diff(e['pos y'])**2)/sf * fps]
# calculate the angle of movement
for e in encounters:
    start_angle = np.arctan2(np.diff(e['pos y'].iloc[:int(fps)]), np.diff(e['pos x'].iloc[:int(fps)]))
    start_angle = np.mean(start_angle) if len(start_angle) > 0 else 0
    e['angle'] = np.r_[start_angle, np.arctan2(np.diff(e['pos y']), np.diff(e['pos x']))]
    e['angle'] = np.unwrap(e['angle'])
    e['angular speed'] = np.abs(np.r_[0, np.diff(e['angle']) * fps])
    e['big ring dist'] = e['big ring dist'] / sf
    e['small ring dist'] = e['small ring dist'] / sf
    
# combine all encounters
encounters_ = pd.concat(encounters, axis=0)

# we define the speed threshold 
speed_threshold = 4 # mm/s

# # plot the distribution of speeds
# plt.figure()
# sns.histplot(encounters_['speed'], bins=50, color='white', edgecolor='black')
# plt.axvline(speed_threshold, color='red', linestyle='--')
# plt.xlabel('Speed (mm/s)')
# plt.ylabel('')
# plt.gca().spines['top'].set_visible(False)
# plt.gca().spines['right'].set_visible(False)
# plt.gca().spines['left'].set_color('black')
# plt.yticks([])
# plt.yscale('log')
# plt.show()

for e in encounters:
    e['moving'] = e['speed'] > speed_threshold
    e['movement speed'] = np.where(e['moving'], e['speed'], np.nan)
    e['moving angular speed'] = np.where(e['moving'], e['angular speed'], np.nan)

encounters = pd.concat(encounters, axis=0)


# get speed as a function of the distance from the centerline
plt.figure(figsize=(2.5, 4))
# add a binned average
bins = np.linspace(np.floor(encounters[f'{ring} ring dist'].min()), np.ceil(encounters[f'{ring} ring dist'].max()), 40)
binned_speed_mean = [encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['moving'].mean() for i in range(bins.size-1)]
binned_speed_se = [encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['moving'].std() for i in range(bins.size-1)] / np.sqrt([encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['speed'].size for i in range(bins.size-1)])
plt.plot(bins[:-1]+np.diff(bins)/2, binned_speed_mean, color='white', linewidth=2)
plt.fill_between(bins[:-1]+np.diff(bins)/2, np.array(binned_speed_mean)-np.array(binned_speed_se), np.array(binned_speed_mean)+np.array(binned_speed_se), color='white', alpha=0.3)
# plot trail width
plt.fill_betweenx([0, 1], -ring_props['trail_width']/2/sf, ring_props['trail_width']/2/sf, color='red', alpha=0.3, zorder=-1)
plt.ylim(0, 1) 
# plt.xlabel('Distance from centerline (mm)')
plt.ylabel('p(moving)', fontsize=14)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.tight_layout()
plt.show()

# get speed as a function of the distance from the centerline
plt.figure(figsize=(2.5, 4))
plt.scatter(encounters[f'{ring} ring dist'], encounters['movement speed'], color='gray', alpha=0.2, s=1)
# add a binned average
bins = np.linspace(np.floor(encounters[f'{ring} ring dist'].min()), np.ceil(encounters[f'{ring} ring dist'].max()), 40)
binned_speed_mean = [encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['movement speed'].mean() for i in range(bins.size-1)]
binned_speed_se = [encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['movement speed'].std() for i in range(bins.size-1)] / np.sqrt([encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['speed'].size for i in range(bins.size-1)])
plt.plot(bins[:-1]+np.diff(bins)/2, binned_speed_mean, color='white', linewidth=2)
plt.fill_between(bins[:-1]+np.diff(bins)/2, np.array(binned_speed_mean)-np.array(binned_speed_se), np.array(binned_speed_mean)+np.array(binned_speed_se), color='white', alpha=0.3)
# plot trail width
plt.fill_betweenx([5, 20], -ring_props['trail_width']/2/sf, ring_props['trail_width']/2/sf, color='red', alpha=0.3, zorder=-1)
plt.ylim(5, 20) 
# plt.xlabel('Distance from centerline (mm)')
plt.ylabel('Speed (mm/s)', fontsize=14)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.tight_layout()
plt.show()

# get angular speed as a function of the distance from the centerline
plt.figure(figsize=(2.5, 4))
plt.scatter(encounters[f'{ring} ring dist'], encounters['moving angular speed'], color='gray', alpha=0.2, s=1)
# add a binned average
bins = np.linspace(np.floor(encounters[f'{ring} ring dist'].min()), np.ceil(encounters[f'{ring} ring dist'].max()), 40)
binned_speed_mean = [encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['moving angular speed'].mean() for i in range(bins.size-1)]
binned_speed_se = [encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['moving angular speed'].std() for i in range(bins.size-1)] / np.sqrt([encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['angular speed'].size for i in range(bins.size-1)])
plt.plot(bins[:-1]+np.diff(bins)/2, binned_speed_mean, color='white', linewidth=2)
plt.fill_between(bins[:-1]+np.diff(bins)/2, np.array(binned_speed_mean)-np.array(binned_speed_se), np.array(binned_speed_mean)+np.array(binned_speed_se), color='white', alpha=0.3)
# plot trail width
plt.fill_betweenx([0, 32], -ring_props['trail_width']/2/sf, ring_props['trail_width']/2/sf, color='red', alpha=0.3, zorder=-1)
plt.ylim(0, 32)
# plt.xlabel('Distance from centerline (mm)')
plt.ylabel('Angular speed (rad/s)', fontsize=14)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.tight_layout()
plt.show()

In [957]:
index, phase, ring = 3, 1, 'big'
encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
encounters = [group for _, group in encounters.groupby('encounter id')]
fps = int(EXP_DB['fps'].iloc[index].astype(float))
sf = EXP_DB['sf'].iloc[index].astype(float)
ring_props = json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r'))

# calculate the speed
for e in encounters:
    start_speed = np.sqrt(np.diff(e['pos x'].iloc[:int(fps)])**2 + np.diff(e['pos y'].iloc[:int(fps)])**2)/sf * fps
    start_speed = np.mean(start_speed) if len(start_speed) > 0 else 0
    e['speed'] = np.r_[start_speed, np.sqrt(np.diff(e['pos x'])**2 + np.diff(e['pos y'])**2)/sf * fps]
# calculate the angle of movement
for e in encounters:
    start_angle = np.arctan2(np.diff(e['pos y'].iloc[:int(fps)]), np.diff(e['pos x'].iloc[:int(fps)]))
    start_angle = np.mean(start_angle) if len(start_angle) > 0 else 0
    e['angle'] = np.r_[start_angle, np.arctan2(np.diff(e['pos y']), np.diff(e['pos x']))]
    e['angle'] = np.unwrap(e['angle'])
    e['angular speed'] = np.abs(np.r_[0, np.diff(e['angle']) * fps])
    e['big ring dist'] = e['big ring dist'] / sf
    e['small ring dist'] = e['small ring dist'] / sf
    
# combine all encounters
encounters_ = pd.concat(encounters, axis=0)

# we define the speed threshold 
speed_threshold = 4 # mm/s

# # plot the distribution of speeds
# plt.figure()
# sns.histplot(encounters_['speed'], bins=50, color='white', edgecolor='black')
# plt.axvline(speed_threshold, color='red', linestyle='--')
# plt.xlabel('Speed (mm/s)')
# plt.ylabel('')
# plt.gca().spines['top'].set_visible(False)
# plt.gca().spines['right'].set_visible(False)
# plt.gca().spines['left'].set_color('black')
# plt.yticks([])
# plt.yscale('log')
# plt.show()

for e in encounters:
    e['moving'] = e['speed'] > speed_threshold
    e['movement speed'] = np.where(e['moving'], e['speed'], np.nan)
    e['moving angular speed'] = np.where(e['moving'], e['angular speed'], np.nan)

encounters = pd.concat(encounters, axis=0)


# get speed as a function of the distance from the centerline
plt.figure(figsize=(2.5, 4))
# add a binned average
bins = np.linspace(np.floor(encounters[f'{ring} ring dist'].min()), np.ceil(encounters[f'{ring} ring dist'].max()), 40)
binned_speed_mean = [encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['moving'].mean() for i in range(bins.size-1)]
binned_speed_se = [encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['moving'].std() for i in range(bins.size-1)] / np.sqrt([encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['speed'].size for i in range(bins.size-1)])
plt.plot(bins[:-1]+np.diff(bins)/2, binned_speed_mean, color='white', linewidth=2)
plt.fill_between(bins[:-1]+np.diff(bins)/2, np.array(binned_speed_mean)-np.array(binned_speed_se), np.array(binned_speed_mean)+np.array(binned_speed_se), color='white', alpha=0.3)
# plot trail width
plt.fill_betweenx([0, 1], -ring_props['trail_width']/2/sf, ring_props['trail_width']/2/sf, color='red', alpha=0.3, zorder=-1)
plt.ylim(0, 1) 
# plt.xlabel('Distance from centerline (mm)')
plt.ylabel('p(moving)', fontsize=14)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.tight_layout()
plt.show()

# get speed as a function of the distance from the centerline
plt.figure(figsize=(2.5, 4))
plt.scatter(encounters[f'{ring} ring dist'], encounters['movement speed'], color='gray', alpha=0.2, s=1)
# add a binned average
bins = np.linspace(np.floor(encounters[f'{ring} ring dist'].min()), np.ceil(encounters[f'{ring} ring dist'].max()), 40)
binned_speed_mean = [encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['movement speed'].mean() for i in range(bins.size-1)]
binned_speed_se = [encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['movement speed'].std() for i in range(bins.size-1)] / np.sqrt([encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['speed'].size for i in range(bins.size-1)])
plt.plot(bins[:-1]+np.diff(bins)/2, binned_speed_mean, color='white', linewidth=2)
plt.fill_between(bins[:-1]+np.diff(bins)/2, np.array(binned_speed_mean)-np.array(binned_speed_se), np.array(binned_speed_mean)+np.array(binned_speed_se), color='white', alpha=0.3)
# plot trail width
plt.fill_betweenx([5, 20], -ring_props['trail_width']/2/sf, ring_props['trail_width']/2/sf, color='red', alpha=0.3, zorder=-1)
plt.ylim(5, 20) 
# plt.xlabel('Distance from centerline (mm)')
plt.ylabel('Speed (mm/s)', fontsize=14)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.tight_layout()
plt.show()

# get angular speed as a function of the distance from the centerline
plt.figure(figsize=(2.5, 4))
plt.scatter(encounters[f'{ring} ring dist'], encounters['moving angular speed'], color='gray', alpha=0.2, s=1)
# add a binned average
bins = np.linspace(np.floor(encounters[f'{ring} ring dist'].min()), np.ceil(encounters[f'{ring} ring dist'].max()), 40)
binned_speed_mean = [encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['moving angular speed'].mean() for i in range(bins.size-1)]
binned_speed_se = [encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['moving angular speed'].std() for i in range(bins.size-1)] / np.sqrt([encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['angular speed'].size for i in range(bins.size-1)])
plt.plot(bins[:-1]+np.diff(bins)/2, binned_speed_mean, color='white', linewidth=2)
plt.fill_between(bins[:-1]+np.diff(bins)/2, np.array(binned_speed_mean)-np.array(binned_speed_se), np.array(binned_speed_mean)+np.array(binned_speed_se), color='white', alpha=0.3)
# plot trail width
plt.fill_betweenx([0, 32], -ring_props['trail_width']/2/sf, ring_props['trail_width']/2/sf, color='red', alpha=0.3, zorder=-1)
plt.ylim(0, 32)
# plt.xlabel('Distance from centerline (mm)')
plt.ylabel('Angular speed (rad/s)', fontsize=14)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.tight_layout()
plt.show()

/var/folders/yj/5_b3w2q176x67w94977kdrx80000gn/T/ipykernel_79510/1865374141.py:2: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')


#### Looking at the top encounters

#### ORCO

In [897]:
index, phase, ring = 2, 1, 'big'
encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
encounters = [group for _, group in encounters.groupby('encounter id')]
# remove encounters with no entries
encounters = [e for e in encounters if e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] > 0]
sort_index_displacement = np.argsort([e['displacement'].iloc[0] for e in encounters])[::-1]
sort_index_max_distance = np.argsort([e['maximum distance'].iloc[0] for e in encounters])[::-1]
# get the top 25 from each and get the common ones
top_displacement = [i for i in sort_index_displacement[:30]]
top_max_distance = [i for i in sort_index_max_distance[:30]]
top_encounters = list(set(top_displacement).intersection(top_max_distance))
# sort by maximum distance
sort_index = np.argsort([encounters[i]['displacement'].iloc[0] for i in top_encounters])[::-1]
top_encounters = [top_encounters[i] for i in sort_index]
print(len(top_encounters))

fps = int(EXP_DB['fps'].iloc[index].astype(float))
sf = EXP_DB['sf'].iloc[index].astype(float)
ring_props = json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r'))

19


In [898]:
# plot the top 10 encounters
fig, ax = plt.subplots(9,1,figsize=(10, 10), sharex=True)
for i, e in enumerate(top_encounters[:9]):
    # plt.plot(encounters[e]['encounter position'], encounters[e][f'{ring} ring dist']/sf, color='white', alpha=0.5, linewidth=0.5)
    ax[i].plot(encounters[e]['encounter position'], encounters[e][f'{ring} ring dist']/sf, color='white', alpha=0.5, linewidth=1)
    ax[i].fill_between([encounters[e]['encounter position'].min(), encounters[e]['encounter position'].max()], -ring_props['trail_width']/2/sf, ring_props['trail_width']/2/sf, color='red', alpha=0.15) 
    ax[i].fill_between([encounters[e]['encounter position'].min(), encounters[e]['encounter position'].max()], -ring_props['trail_width']/sf, ring_props['trail_width']/sf, color='red', alpha=0.15) 
    # ax[i].set_xlim(encounters[e]['encounter position'].min(), encounters[e]['encounter position'].max())
    ax[i].invert_yaxis()
    ax[i].spines['top'].set_visible(False)
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['bottom'].set_visible(False)
    if i == 4:
        ax[i].set_ylabel('Distance from centerline (mm)')
    if i == 8:
        ax[i].set_xlabel('Position along the band (mm)')
    else:
        # change x tick marker color
        ax[i].tick_params(axis='x', colors='black')
        
plt.show()


In [893]:
# make a video of the top 10 encounters
prefix = EXP_DB['data_directory'].iloc[index].split('/')[-2]
p1_bg = sio.loadmat(f'{EXP_DB["data_directory"].iloc[index]}/processed/{prefix}_phase_1/{prefix}_phase_1-bg.mat')['bg'][0][0].item()[0]
downsample = 2
speed_up = 8

# downsample the background
bg = p1_bg[::downsample, ::downsample]
# get the size of the background
bg_size_or = bg.shape
# repeat the background in a 3x3 grid
bg = (np.tile(bg, (3,3))*255).astype(np.uint8)
# get the size of the background
bg_size = bg.shape

# add text to the background
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(bg, 'Orco-Gal4/UAS-CsChrimson', (20, 40), font, 1.5, (255, 255, 255), 1, cv2.LINE_AA)
cv2.putText(bg, f'Speed: {speed_up}x', (20, 80), font, 1.5, (255, 255, 255), 1, cv2.LINE_AA)

# get the durations of the encounters
durations = [len(encounters[e]) for e in top_encounters[:9]]

# use opencv to draw the trajectories in mp4
import cv2
from tqdm import tqdm
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter('top_encounters_band-orco.mp4', fourcc, fps*speed_up, (bg_size[1], bg_size[0]))

for t in tqdm(range(max(durations))):
    frame = bg.copy()
    # convert the frame to color
    frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
    for i, e in enumerate(top_encounters[:9]):
        if t < len(encounters[e]):
            x = (encounters[e]['pos x'].iloc[:t+1].values/downsample).astype(int)
            y = (encounters[e]['pos y'].iloc[:t+1].values/downsample).astype(int)
            # offset the position
            x += bg_size_or[1]*(i//3)
            y += bg_size_or[0]*(i%3)
            cv2.circle(frame, (x[-1], y[-1]), 4, (0, 0, 255), -1)
            # draw the trajectory
            if t > 0:
                cv2.line(bg, (x[-2], y[-2]), (x[-1], y[-1]), (255, 255, 255), 1)
    video.write(frame)
video.release()

100%|██████████| 1228/1228 [00:14<00:00, 82.75it/s] 


In [899]:
index, phase, ring = 3, 1, 'big'
encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
encounters = [group for _, group in encounters.groupby('encounter id')]
# remove encounters with no entries
encounters = [e for e in encounters if e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] > 0]
sort_index_displacement = np.argsort([e['displacement'].iloc[0] for e in encounters])[::-1]
sort_index_max_distance = np.argsort([e['maximum distance'].iloc[0] for e in encounters])[::-1]
# get the top 25 from each and get the common ones
top_displacement = [i for i in sort_index_displacement[:30]]
top_max_distance = [i for i in sort_index_max_distance[:30]]
top_encounters = list(set(top_displacement).intersection(top_max_distance))
# sort by maximum distance
sort_index = np.argsort([encounters[i]['displacement'].iloc[0] for i in top_encounters])[::-1]
top_encounters = [top_encounters[i] for i in sort_index]
print(len(top_encounters))

fps = int(EXP_DB['fps'].iloc[index].astype(float))
sf = EXP_DB['sf'].iloc[index].astype(float)
ring_props = json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r'))

/var/folders/yj/5_b3w2q176x67w94977kdrx80000gn/T/ipykernel_79510/4126001089.py:2: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')


18


In [900]:
# plot the top 10 encounters
fig, ax = plt.subplots(9,1,figsize=(10, 10), sharex=True)
for i, e in enumerate(top_encounters[:9]):
    # plt.plot(encounters[e]['encounter position'], encounters[e][f'{ring} ring dist']/sf, color='white', alpha=0.5, linewidth=0.5)
    ax[i].plot(encounters[e]['encounter position'], encounters[e][f'{ring} ring dist']/sf, color='white', alpha=0.5, linewidth=1)
    ax[i].fill_between([encounters[e]['encounter position'].min(), encounters[e]['encounter position'].max()], -ring_props['trail_width']/2/sf, ring_props['trail_width']/2/sf, color='red', alpha=0.15) 
    ax[i].fill_between([encounters[e]['encounter position'].min(), encounters[e]['encounter position'].max()], -ring_props['trail_width']/sf, ring_props['trail_width']/sf, color='red', alpha=0.15) 
    # ax[i].set_xlim(encounters[e]['encounter position'].min(), encounters[e]['encounter position'].max())
    ax[i].invert_yaxis()
    ax[i].spines['top'].set_visible(False)
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['bottom'].set_visible(False)
    if i == 4:
        ax[i].set_ylabel('Distance from centerline (mm)')
    if i == 8:
        ax[i].set_xlabel('Position along the band (mm)')
    else:
        # change x tick marker color
        ax[i].tick_params(axis='x', colors='black')
plt.show()


In [902]:
# make a video of the top 10 encounters
prefix = EXP_DB['data_directory'].iloc[index].split('/')[-2]
p1_bg = sio.loadmat(f'{EXP_DB["data_directory"].iloc[index]}/processed/{prefix}_phase_1/{prefix}_phase_1-bg.mat')['bg'][0][0].item()[0]
downsample = 3
speed_up = 8

# downsample the background
bg = p1_bg[::downsample, ::downsample]
# get the size of the background
bg_size_or = bg.shape
# repeat the background in a 3x3 grid
bg = (np.tile(bg, (3,3))*255).astype(np.uint8)
# get the size of the background
bg_size = bg.shape
print(bg_size)

# add text to the background
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(bg, 'Orco-Gal4/+', (20, 40), font, 1.5, (255, 255, 255), 1, cv2.LINE_AA)
cv2.putText(bg, f'Speed: {speed_up}x', (20, 80), font, 1.5, (255, 255, 255), 1, cv2.LINE_AA)

# get the durations of the encounters
durations = [len(encounters[e]) for e in top_encounters[:9]]

# use opencv to draw the trajectories in mp4
import cv2
from tqdm import tqdm
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter('top_encounters_band-orcoctrl.mp4', fourcc, fps*speed_up, (bg_size[1], bg_size[0]))

for t in tqdm(range(max(durations))):
    frame = bg.copy()
    # convert the frame to color
    frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
    for i, e in enumerate(top_encounters[:9]):
        if t < len(encounters[e]):
            x = (encounters[e]['pos x'].iloc[:t+1].values/downsample).astype(int)
            y = (encounters[e]['pos y'].iloc[:t+1].values/downsample).astype(int)
            # offset the position
            x += bg_size_or[1]*(i//3)
            y += bg_size_or[0]*(i%3)
            cv2.circle(frame, (x[-1], y[-1]), 4, (0, 0, 255), -1)
            # draw the trajectory
            if t > 0:
                cv2.line(bg, (x[-2], y[-2]), (x[-1], y[-1]), (255, 255, 255), 1)
    video.write(frame)
video.release()

(1398, 1398)


100%|██████████| 554/554 [00:04<00:00, 135.60it/s]


# Other Genotypes

In [80]:
# show only the encounters with 1 entries
def plot_best_encounters(index, phase, ring, color='white', yrange=None, return_yrange=False):
    encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
    encounters = [group for _, group in encounters.groupby('encounter id')]
    # remove encounters with no entries
    encounters = [e for e in encounters if e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] > 2]
    ring_props = json.load(open(EXP_DB[f'{ring}_ring'].iloc[index], 'r'))

    sf = EXP_DB['sf'].iloc[index].astype(float)

    plt.figure(figsize=(3, 10))
    for e in encounters:
        plt.plot(e[f'{ring} ring dist']/sf, e['encounter position'], color='white', alpha=0.5, linewidth=0.5)

    # get the x-axis limits
    if yrange is not None:
        y_min, y_max = yrange
    else:
        y_min = min([e['encounter position'].min() for e in encounters])
        y_max = max([e['encounter position'].max() for e in encounters])

    print(f'n = {len(encounters)} encounters ({EXP_DB["n_flies"].iloc[index]} flies)')
    # fill the trail
    plt.fill_betweenx([y_min, y_max], -ring_props['trail_width']/2/sf, ring_props['trail_width']/2/sf, color=color, alpha=0.3)
    plt.ylim(y_min, y_max)
    plt.xticks([-15, 0, 15])
    plt.gca().set_aspect('equal')
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_color('black')
    if not os.path.exists(f'{EXP_DB["processed_data_directory"].iloc[index]}/figures'):
        os.makedirs(f'{EXP_DB["processed_data_directory"].iloc[index]}/figures')
    plt.savefig(f'{EXP_DB["processed_data_directory"].iloc[index]}/figures/p{phase}_{ring}_ring_best_encounters.png', dpi=300)
    plt.show()

    if return_yrange:
        return y_min, y_max

In [81]:
EXP_DB

,n_flies,n_frames1,n_frames2,n_frames3,sf,fps,data_directory,processed_data_directory,arena,big_ring,small_ring,starvation,wings?,genotype,date,time
0,5,36000,18000,35954,5.535856,10.0,../data/data_06-09-2024/Or42B/Band/20hr-wingle...,../processed_data/data_06-09-2024/Or42B/Band/2...,../processed_data/data_06-09-2024/Or42B/Band/2...,../processed_data/data_06-09-2024/Or42B/Band/2...,../processed_data/data_06-09-2024/Or42B/Band/2...,20hr,wingless,f,2024-04-18,11-38
1,5,36000,18000,35924,5.525171,10.0,../data/data_06-09-2024/Or42B/Band/23hr-wingle...,../processed_data/data_06-09-2024/Or42B/Band/2...,../processed_data/data_06-09-2024/Or42B/Band/2...,../processed_data/data_06-09-2024/Or42B/Band/2...,../processed_data/data_06-09-2024/Or42B/Band/2...,23hr,wingless,f,2024-04-18,15-25
2,5,36000,18000,35948,5.518664,10.0,../data/data_06-09-2024/OrCo/Band/19hr-wingles...,../processed_data/data_06-09-2024/OrCo/Band/19...,../processed_data/data_06-09-2024/OrCo/Band/19...,../processed_data/data_06-09-2024/OrCo/Band/19...,../processed_data/data_06-09-2024/OrCo/Band/19...,19hr,wingless,orco,2024-04-17,11-17
3,4,36000,18000,35967,9.287088,10.0,../data/data_06-09-2024/OrCo/Band/20hr-wingles...,../processed_data/data_06-09-2024/OrCo/Band/20...,../processed_data/data_06-09-2024/OrCo/Band/20...,../processed_data/data_06-09-2024/OrCo/Band/20...,../processed_data/data_06-09-2024/OrCo/Band/20...,20hr,wingless,orcoctrl,2024-05-29,11-29


In [82]:
# look at all genotypes
y_range = plot_best_encounters(2, 1, 'big', color='red', return_yrange=True)
plot_best_encounters(2, 2, 'big', color='gray', yrange=y_range)
plot_best_encounters(2, 3, 'small', color='red', yrange=y_range)
plot_best_encounters(3, 1, 'big', color='red', yrange=y_range)
plot_best_encounters(3, 2, 'big', color='gray', yrange=y_range)
plot_best_encounters(3, 3, 'small', color='red', yrange=y_range)

n = 51 encounters (5 flies)
n = 12 encounters (5 flies)
n = 25 encounters (5 flies)


/var/folders/yj/5_b3w2q176x67w94977kdrx80000gn/T/ipykernel_66784/1675443245.py:3: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')


n = 13 encounters (4 flies)
n = 2 encounters (4 flies)
n = 1 encounters (4 flies)


In [83]:
print(EXP_DB['processed_data_directory'].iloc[0])
plot_best_encounters(0, 1, 'big', color='red', yrange=y_range)
plot_best_encounters(0, 2, 'big', color='gray', yrange=y_range)
plot_best_encounters(0, 3, 'small', color='red', yrange=y_range)

print(EXP_DB['processed_data_directory'].iloc[1])
plot_best_encounters(1, 1, 'big', color='red', yrange=y_range)
plot_best_encounters(1, 2, 'big', color='gray', yrange=y_range)
plot_best_encounters(1, 3, 'small', color='red', yrange=y_range)

../processed_data/data_06-09-2024/Or42B/Band/20hr-wingless-f-or42b-band/2024-04-18_11-38


/var/folders/yj/5_b3w2q176x67w94977kdrx80000gn/T/ipykernel_66784/1675443245.py:3: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')


n = 27 encounters (5 flies)
n = 6 encounters (5 flies)


/var/folders/yj/5_b3w2q176x67w94977kdrx80000gn/T/ipykernel_66784/1675443245.py:3: DtypeWarning: Columns (22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')


n = 26 encounters (5 flies)
../processed_data/data_06-09-2024/Or42B/Band/23hr-wingless-f-or42b-band/2024-04-18_15-25
n = 31 encounters (5 flies)
n = 14 encounters (5 flies)
n = 31 encounters (5 flies)


In [87]:
def get_encounter_props(index, phase, ring, genotype):
    encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
    encounters = [group for _, group in encounters.groupby('encounter id')]
    # keep only encounters with more than 2 entries
    encounters = [e for e in encounters if e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] > 2]
    df = pd.DataFrame(get_props(encounters))
    df['genotype'] = genotype
    return df.reset_index(drop=True)

from scipy.stats import mannwhitneyu

# orco
index, phase, ring = 2, 1, 'big'
orco_df = get_encounter_props(index, phase, ring, 'Orco-Gal4/UAS-CsChrimson')
# or42b
index, phase, ring = 0, 1, 'big'
or42b_df1 = get_encounter_props(index, phase, ring, 'Or42b-Gal4/UAS-CsChrimson')
index, phase, ring = 1, 1, 'big'
or42b_df2 = get_encounter_props(index, phase, ring, 'Or42b-Gal4/UAS-CsChrimson')
or42b_df = pd.concat([or42b_df1, or42b_df2], axis=0)
# orcoctrl
index, phase, ring = 3, 1, 'big'
orcoctrl_df = get_encounter_props(index, phase, ring, 'Orco-Gal4/+')
# combine all dataframes
df = pd.concat([orco_df,or42b_df, orcoctrl_df], axis=0).reset_index(drop=True)


for props in ['duration (s)', 'mean speed (mm/s)', 'fraction in trail', 'max distance (mm)', 'inverse tortuosity', 'displacement (mm)']:
    plt.figure(figsize=(4, 4))
    
    sns.stripplot(x='genotype', y=props, data=df, palette=['white', 'orange', 'gray'], alpha=1, size=5, hue='genotype')
    # show the mean line
    sns.pointplot(x='genotype', y=props, data=df, color='darkgray', markers='_', errorbar=('ci', 95), capsize=0.1, zorder=10, linewidth=1)

    # compare the middle genotype to the other two genotypes
    p1 = mannwhitneyu(orco_df[props], or42b_df[props]).pvalue
    p2 = mannwhitneyu(or42b_df[props], orcoctrl_df[props]).pvalue
    plt.text(0.5, df[props].max()*0.9, f'p={p1:.3f}' if p1 > 0.001 else 'p<0.001', ha='center', va='top', fontsize=8)
    plt.text(0.5, df[props].max()*0.95, p_value_to_stars(p1), ha='center', va='top', fontsize=12)
    plt.text(1.5, df[props].max()*0.9, f'p={p2:.3f}' if p2 > 0.001 else 'p<0.001', ha='center', va='top', fontsize=8)
    plt.text(1.5, df[props].max()*0.95, p_value_to_stars(p2), ha='center', va='top', fontsize=12)


    plt.xticks([0, 1, 2], ['Orco-Gal4/\nUAS-CsChrimson', 'Or42b-Gal4/\nUAS-CsChrimson', 'Orco-Gal4/+'])
    plt.ylabel(props)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_color('black')
    plt.tight_layout()
    # remove the legend
    plt.gca().get_legend().remove()
    plt.show()

for props in ['center crossings', 'total entries']:
    plt.figure(figsize=(4, 4))
    
    sns.histplot(x='genotype', y=props, data=df, palette=['white', 'orange','gray'], alpha=1, hue='genotype', legend=False)

    # compare the middle genotype to the other two genotypes
    p1 = mannwhitneyu(orco_df[props], or42b_df[props]).pvalue
    p2 = mannwhitneyu(or42b_df[props], orcoctrl_df[props]).pvalue
    plt.text(0.5, df[props].max()*1.05, f'p={p1:.3f}' if p1 > 0.001 else 'p<0.001', ha='center', va='top', fontsize=8)
    plt.text(0.5, df[props].max()*1.10, p_value_to_stars(p1), ha='center', va='top', fontsize=12)
    plt.text(1.5, df[props].max()*1.05, f'p={p2:.3f}' if p2 > 0.001 else 'p<0.001', ha='center', va='top', fontsize=8)
    plt.text(1.5, df[props].max()*1.10, p_value_to_stars(p2), ha='center', va='top', fontsize=12)


    plt.xticks([0, 1, 2], ['Orco-Gal4/\nUAS-CsChrimson', 'Or42b-Gal4/\nUAS-CsChrimson', 'Orco-Gal4/+'])
    plt.ylabel(props)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_color('black')
    plt.tight_layout()
    plt.show()

/var/folders/yj/5_b3w2q176x67w94977kdrx80000gn/T/ipykernel_66784/2754752589.py:2: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
/var/folders/yj/5_b3w2q176x67w94977kdrx80000gn/T/ipykernel_66784/2754752589.py:2: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[index] + f'/p{phase}_{ring}_ring_encounters.csv')
/Users/neurorishika/mambaforge3/lib/python3.10/site-packages/seaborn/categorical.py:1733: UserWarning: You passed a edgecolor/edgecolors ([(0.6627450980392157, 0.6627450980392157, 0.6627450980392157), (0.6627450980392157, 0.6627450980392157, 0.6627450980392157), (0.6627450980392157, 0.6627450980392157, 0.6627450980392157)]) for an unfilled marker ('_').  Matplotlib is ignoring the edgecolor in fav

In [96]:
index, phase, ring = [0,1], 1, 'big'

# we define the speed threshold 
speed_threshold = 4 # mm/s

all_encounters = []
for i in index:
    encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[i] + f'/p{phase}_{ring}_ring_encounters.csv')
    encounters = [group for _, group in encounters.groupby('encounter id')]
    # remove encounters with no entries
    encounters = [e for e in encounters if e['num inner entries'].iloc[0]+e['num outer entries'].iloc[0] > 0]
    fps = int(EXP_DB['fps'].iloc[i].astype(float))
    sf = EXP_DB['sf'].iloc[i].astype(float)
    ring_props = json.load(open(EXP_DB[f'{ring}_ring'].iloc[i], 'r'))
    # calculate the speed
    for e in encounters:
        start_speed = np.sqrt(np.diff(e['pos x'].iloc[:int(fps)])**2 + np.diff(e['pos y'].iloc[:int(fps)])**2)/sf * fps
        start_speed = np.mean(start_speed) if len(start_speed) > 0 else 0
        e['speed'] = np.r_[start_speed, np.sqrt(np.diff(e['pos x'])**2 + np.diff(e['pos y'])**2)/sf * fps]
        start_angle = np.arctan2(np.diff(e['pos y'].iloc[:int(fps)]), np.diff(e['pos x'].iloc[:int(fps)]))
        start_angle = np.mean(start_angle) if len(start_angle) > 0 else 0
        e['angle'] = np.r_[start_angle, np.arctan2(np.diff(e['pos y']), np.diff(e['pos x']))]
        e['angle'] = np.unwrap(e['angle'])
        e['angular speed'] = np.abs(np.r_[0, np.diff(e['angle']) * fps])
        e['big ring dist'] = e['big ring dist'] / sf
        e['small ring dist'] = e['small ring dist'] / sf
        e['moving'] = e['speed'] > speed_threshold
        e['movement speed'] = np.where(e['moving'], e['speed'], np.nan)
        e['moving angular speed'] = np.where(e['moving'], e['angular speed'], np.nan)
    all_encounters+=encounters

encounters = all_encounters

/var/folders/yj/5_b3w2q176x67w94977kdrx80000gn/T/ipykernel_66784/594941597.py:8: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  encounters = pd.read_csv(EXP_DB['processed_data_directory'].iloc[i] + f'/p{phase}_{ring}_ring_encounters.csv')


In [97]:


# calculate the mean speed and heading before the first entry
mean_speeds = []
mean_headings = []
max_distances = []
displacements = []
for e in encounters:
    e['speed'] = np.r_[0, np.sqrt(np.diff(e['pos x'])**2 + np.diff(e['pos y'])**2)/sf * fps]
    e['angle'] = np.r_[0, np.arctan2(np.diff(e['pos y']), np.diff(e['pos x']))]
    e['angle'] = e['angle'] % (2*np.pi)
    e['adjusted angle'] = (e['angle'] - e[f'{ring} ring pos']% (2*np.pi)) % (2*np.pi)
    # get first entry
    inner_entries = str(e['inner entries'].iloc[0]).split(',') if str(e['inner entries'].iloc[0]) != 'nan' else []
    outer_entries = str(e['outer entries'].iloc[0]).split(',') if str(e['outer entries'].iloc[0]) != 'nan' else []
    entries = inner_entries + outer_entries
    first_entry = np.min([int(e.split('.')[0]) for e in entries])
    mean_speeds.append(e['speed'].iloc[:first_entry].mean())
    mean_headings.append(e['adjusted angle'].iloc[:first_entry].mean())
    max_distances.append(e['maximum distance'].iloc[0])
    displacements.append(e['displacement'].iloc[0])

In [98]:
# plot the maximum distance as a function of the mean speed
plt.figure()
plt.scatter(mean_speeds, max_distances, color='white', edgecolor='black', s=15)
# binned average
bins = np.linspace(np.floor(min(mean_speeds)), np.ceil(max(mean_speeds)), 10)
binned_max_distance_mean = [np.mean([max_distances[i] for i in range(len(mean_speeds)) if mean_speeds[i] >= bins[j] and mean_speeds[i] < bins[j+1]]) for j in range(bins.size-1)]
binned_max_distance_se = [np.std([max_distances[i] for i in range(len(mean_speeds)) if mean_speeds[i] >= bins[j] and mean_speeds[i] < bins[j+1]]) / np.sqrt(len([max_distances[i] for i in range(len(mean_speeds)) if mean_speeds[i] >= bins[j] and mean_speeds[i] < bins[j+1]])) for j in range(bins.size-1)]
plt.plot(bins[:-1]+np.diff(bins)/2, binned_max_distance_mean, color='white', linewidth=2)
plt.fill_between(bins[:-1]+np.diff(bins)/2, np.array(binned_max_distance_mean)-np.array(binned_max_distance_se), np.array(binned_max_distance_mean)+np.array(binned_max_distance_se), color='white', alpha=0.3)
plt.xlabel('Mean speed (mm/s)')
plt.ylabel('Maximum distance (mm)')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.show()

# plot the displacement as a function of the mean speed
plt.figure()
plt.scatter(mean_speeds, displacements, color='white', edgecolor='black', s=15)
# binned average
bins = np.linspace(np.floor(min(mean_speeds)), np.ceil(max(mean_speeds)), 10)
binned_displacement_mean = [np.mean([displacements[i] for i in range(len(mean_speeds)) if mean_speeds[i] >= bins[j] and mean_speeds[i] < bins[j+1]]) for j in range(bins.size-1)]
binned_displacement_se = [np.std([displacements[i] for i in range(len(mean_speeds)) if mean_speeds[i] >= bins[j] and mean_speeds[i] < bins[j+1]]) / np.sqrt(len([displacements[i] for i in range(len(mean_speeds)) if mean_speeds[i] >= bins[j] and mean_speeds[i] < bins[j+1]])) for j in range(bins.size-1)]
plt.plot(bins[:-1]+np.diff(bins)/2, binned_displacement_mean, color='white', linewidth=2)
plt.fill_between(bins[:-1]+np.diff(bins)/2, np.array(binned_displacement_mean)-np.array(binned_displacement_se), np.array(binned_displacement_mean)+np.array(binned_displacement_se), color='white', alpha=0.3)
plt.xlabel('Mean speed (mm/s)')
plt.ylabel('Displacement (mm)')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.show()

In [99]:
# plot the maximum distance as a function of the mean heading
plt.figure()
plt.scatter(mean_headings, max_distances, color='white', edgecolor='black', s=15)
# binned average
bins = np.arange(0, 2*np.pi+0.1, np.pi/4)
binned_max_distance_mean = [np.mean([max_distances[i] for i in range(len(mean_headings)) if mean_headings[i] >= bins[j] and mean_headings[i] < bins[j+1]]) for j in range(bins.size-1)]
binned_max_distance_se = [np.std([max_distances[i] for i in range(len(mean_headings)) if mean_headings[i] >= bins[j] and mean_headings[i] < bins[j+1]]) / np.sqrt(len([max_distances[i] for i in range(len(mean_headings)) if mean_headings[i] >= bins[j] and mean_headings[i] < bins[j+1]])) for j in range(bins.size-1)]
plt.plot(bins[:-1]+np.diff(bins)/2, binned_max_distance_mean, color='white', linewidth=2)
plt.fill_between(bins[:-1]+np.diff(bins)/2, np.array(binned_max_distance_mean)-np.array(binned_max_distance_se), np.array(binned_max_distance_mean)+np.array(binned_max_distance_se), color='white', alpha=0.3)
plt.xlabel('Mean heading (rad)')
plt.xticks( np.arange(0, 2*np.pi+0.1, np.pi/2), ['0', r'$\pi/2$', r'$\pi$', r'$3\pi/2$', r'$2\pi$'])
plt.ylabel('Maximum distance (mm)')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.show()

# plot the displacement as a function of the mean heading
plt.figure()
plt.scatter(mean_headings, displacements, color='white', edgecolor='black', s=15)
# binned average
bins = np.arange(0, 2*np.pi+0.1, np.pi/4)
binned_displacement_mean = [np.mean([displacements[i] for i in range(len(mean_headings)) if mean_headings[i] >= bins[j] and mean_headings[i] < bins[j+1]]) for j in range(bins.size-1)]
binned_displacement_se = [np.std([displacements[i] for i in range(len(mean_headings)) if mean_headings[i] >= bins[j] and mean_headings[i] < bins[j+1]]) / np.sqrt(len([displacements[i] for i in range(len(mean_headings)) if mean_headings[i] >= bins[j] and mean_headings[i] < bins[j+1]])) for j in range(bins.size-1)]
plt.plot(bins[:-1]+np.diff(bins)/2, binned_displacement_mean, color='white', linewidth=2)
plt.fill_between(bins[:-1]+np.diff(bins)/2, np.array(binned_displacement_mean)-np.array(binned_displacement_se), np.array(binned_displacement_mean)+np.array(binned_displacement_se), color='white', alpha=0.3)
plt.xlabel('Mean heading (rad)')
plt.xticks( np.arange(0, 2*np.pi+0.1, np.pi/2), ['0', r'$\pi/2$', r'$\pi$', r'$3\pi/2$', r'$2\pi$'])
plt.ylabel('Displacement (mm)')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.show()




In [100]:
encounters = pd.concat(encounters, axis=0)


# get speed as a function of the distance from the centerline
plt.figure(figsize=(2.5, 4))
# add a binned average
bins = np.linspace(np.floor(encounters[f'{ring} ring dist'].min()), np.ceil(encounters[f'{ring} ring dist'].max()), 40)
binned_speed_mean = [encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['moving'].mean() for i in range(bins.size-1)]
binned_speed_se = [encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['moving'].std() for i in range(bins.size-1)] / np.sqrt([encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['speed'].size for i in range(bins.size-1)])
plt.plot(bins[:-1]+np.diff(bins)/2, binned_speed_mean, color='white', linewidth=2)
plt.fill_between(bins[:-1]+np.diff(bins)/2, np.array(binned_speed_mean)-np.array(binned_speed_se), np.array(binned_speed_mean)+np.array(binned_speed_se), color='white', alpha=0.3)
# plot trail width
plt.fill_betweenx([0, 1], -ring_props['trail_width']/2/sf, ring_props['trail_width']/2/sf, color='red', alpha=0.3, zorder=-1)
plt.ylim(0, 1) 
# plt.xlabel('Distance from centerline (mm)')
plt.ylabel('p(moving)', fontsize=14)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.tight_layout()
plt.show()

# get speed as a function of the distance from the centerline
plt.figure(figsize=(2.5, 4))
plt.scatter(encounters[f'{ring} ring dist'], encounters['movement speed'], color='gray', alpha=0.2, s=1)
# add a binned average
bins = np.linspace(np.floor(encounters[f'{ring} ring dist'].min()), np.ceil(encounters[f'{ring} ring dist'].max()), 40)
binned_speed_mean = [encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['movement speed'].mean() for i in range(bins.size-1)]
binned_speed_se = [encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['movement speed'].std() for i in range(bins.size-1)] / np.sqrt([encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['speed'].size for i in range(bins.size-1)])
plt.plot(bins[:-1]+np.diff(bins)/2, binned_speed_mean, color='white', linewidth=2)
plt.fill_between(bins[:-1]+np.diff(bins)/2, np.array(binned_speed_mean)-np.array(binned_speed_se), np.array(binned_speed_mean)+np.array(binned_speed_se), color='white', alpha=0.3)
# plot trail width
plt.fill_betweenx([5, 20], -ring_props['trail_width']/2/sf, ring_props['trail_width']/2/sf, color='red', alpha=0.3, zorder=-1)
plt.ylim(5, 20) 
# plt.xlabel('Distance from centerline (mm)')
plt.ylabel('Speed (mm/s)', fontsize=14)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.tight_layout()
plt.show()

# get angular speed as a function of the distance from the centerline
plt.figure(figsize=(2.5, 4))
plt.scatter(encounters[f'{ring} ring dist'], encounters['moving angular speed'], color='gray', alpha=0.2, s=1)
# add a binned average
bins = np.linspace(np.floor(encounters[f'{ring} ring dist'].min()), np.ceil(encounters[f'{ring} ring dist'].max()), 40)
binned_speed_mean = [encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['moving angular speed'].mean() for i in range(bins.size-1)]
binned_speed_se = [encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['moving angular speed'].std() for i in range(bins.size-1)] / np.sqrt([encounters[(encounters[f'{ring} ring dist'] >= bins[i]) & (encounters[f'{ring} ring dist'] < bins[i+1])]['angular speed'].size for i in range(bins.size-1)])
plt.plot(bins[:-1]+np.diff(bins)/2, binned_speed_mean, color='white', linewidth=2)
plt.fill_between(bins[:-1]+np.diff(bins)/2, np.array(binned_speed_mean)-np.array(binned_speed_se), np.array(binned_speed_mean)+np.array(binned_speed_se), color='white', alpha=0.3)
# plot trail width
plt.fill_betweenx([0, 32], -ring_props['trail_width']/2/sf, ring_props['trail_width']/2/sf, color='red', alpha=0.3, zorder=-1)
plt.ylim(0, 32)
# plt.xlabel('Distance from centerline (mm)')
plt.ylabel('Angular speed (rad/s)', fontsize=14)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.tight_layout()
plt.show()

In [791]:
plt.plot(tracklets[tracklets['track id']==0]['angular speed'])

In [ ]:
p2_bg = sio.loadmat(f'{exp}/processed/{prefix}_phase_2/{prefix}_phase_2-bg.mat')['bg'][0][0].item()[0]